## Ensaios sobre previsibilidade nas eleições para vereador
- Trata o presente de ensaios, análises e testes sobre a viabilidade de uso de inteligência artificial para fins de predição das chances de eleição de candidatos para o cargo de vereador, nas eleições municipais
- O foco será a análise com base nas caracteristicas dos candidatos, seus partidos e outras que puderem ser agregadas


#### Bases utilizadas:
- base candidatos do TSE (consulta_cand_AAAA_SP.txt): esta é a base principal, com informações dos candidatos
- base legenda/coligação 
- base quantidade de vagas
- base bens dos condidatos
- base prestação de contas
- base resultados
    - votacao_candidato_munzona_AAAA_SP.txt
    - votacao_partido_munzona_AAAA_SP.txt 
- base filiados
    - população votante do municipio
    - qtd de filiados por partido

In [1]:
# importa bibliotecas que serão usadas no projeto
import os
from zipfile import ZipFile
import shutil
import re
import pandas as pd
import numpy as np
from datetime import datetime
import sweetviz as sv
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# configurações gerais de visualização do jupyter notebook
d_configuracao = {
    'display.max_columns': 150,
    'display.expand_frame_repr': True,
    'display.max_rows': 40,
    'display.precision': 2,
    'display.show_dimensions': True,}

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
for op, value in d_configuracao.items():
    pd.set_option(op, value)
    print(op, value)

display.max_columns 150
display.expand_frame_repr True
display.max_rows 40
display.precision 2
display.show_dimensions True


### 1. Introdução

### 2. Dados utilizados no estudo

### 3. Leitura e tratamento dos dados
#### Fonte principal dos dodos: Repositório de dados do TSE
*https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

#### Observações gerais:
- conforme indicado nos arquivos de apoio (leia.me) do TSE:
    - a codificação dos caracteres dos arquivos é 'latin_1';
    - os campos estão entre aspas "" e separados por ponto e vírgula, inclusive os campos numéricos;
    - O campo UF, além das unidades da federação pode conter alguma das seguintes situações:
        - BR – Quando se tratar de informação a nível nacional;
        - VT – Quando se tratar de voto em trânsito;
        - ZZ – Quando se tratar de Exterior.
    - todas as informações referentes as candidaturas são de responsabilidade do candidato;


### 3.1. Importa dados dos candidatos as eleições
- origem: repositório de dados do TSE
*https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

- tema: "candidatos"
- arquivo: "consulta_cand_2008.zip"
*https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2008.zip


#### O foco do estudo serão as eleições para o legislativo municipal no ano de 2008 e 2012

#### Observações sobre os arquivos de candidatos do TSE
- 3 alterações no layout das informações disponíveis dos candidatos ao longo dos anos:
    - até 2010 (layount base)
    - em 2012 (acrescimo de endereço de e-mail no layout base)
    - a partir de 2014 (diversos acrescimos/ alterações):
    
#### Leitura da base de TESTES modelo (2012):
    - precisamos fazer as mesmas alterações feitas nos dados de treino (2008)
    

In [18]:
# função para leitura de dados da base "candidatos" do TSE
def le_cand(ano, uf):
    # nomes das colunas até 2010
    c_10 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE','NM_UE',
        'CD_CARGO','DS_CARGO','NM_CANDIDATO','SQ_CANDIDATO','NR_CANDIDATO','NR_CPF_CANDIDATO','NM_URNA_CANDIDATO',
        'CD_SITUACAO_CANDIDATURA','DS_SITUACAO_CANDIDATURA','NR_PARTIDO','SG_PARTIDO','NM_PARTIDO','SQ_LEGENDA',
        'SG_LEGENDA','COMPOSICAO_LEGENDA','NM_LEGENDA','CD_OCUPACAO','DS_OCUPACAO','DT_NASCIMENTO',
        'NR_TITULO_ELEITORAL_CANDIDATO','IDADE_DATA_ELEICAO','CD_GENERO','DS_GENERO','CD_GRAU_INSTRUCAO',
        'DS_GRAU_INSTRUCAO','CD_ESTADO_CIVIL','DS_ESTADO_CIVIL','CD_NACIONALIDADE','DS_NACIONALIDADE',
        'SG_UF_NASCIMENTO','CD_MUNICIPIO_NASCIMENTO','NM_MUNICIPIO_NASCIMENTO','VR_DESPESA_MAX_CAMPANHA',
        'CD_SIT_TOT_TURNO','DS_SIT_TOT_TURNO']

    # nomes das colunas em 2012
    c_12 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE','NM_UE',
        'CD_CARGO','DS_CARGO','NM_CANDIDATO','SQ_CANDIDATO','NR_CANDIDATO','NR_CPF_CANDIDATO','NM_URNA_CANDIDATO',
        'CD_SITUACAO_CANDIDATURA','DS_SITUACAO_CANDIDATURA','NR_PARTIDO','SG_PARTIDO','NM_PARTIDO','SQ_LEGENDA',
        'SG_LEGENDA','COMPOSICAO_LEGENDA','NM_LEGENDA','CD_OCUPACAO','DS_OCUPACAO','DT_NASCIMENTO',
        'NR_TITULO_ELEITORAL_CANDIDATO','IDADE_DATA_ELEICAO','CD_GENERO','DS_GENERO','CD_GRAU_INSTRUCAO',
        'DS_GRAU_INSTRUCAO','CD_ESTADO_CIVIL','DS_ESTADO_CIVIL','CD_NACIONALIDADE','DS_NACIONALIDADE',
        'SG_UF_NASCIMENTO','CD_MUNICIPIO_NASCIMENTO','NM_MUNICIPIO_NASCIMENTO','VR_DESPESA_MAX_CAMPANHA',
        'CD_SIT_TOT_TURNO','DS_SIT_TOT_TURNO','NM_EMAIL']
    
    # nome do arquivo zipado, fornecido pelo TSE
    nome_zip = 'tse/candidatos/consulta_cand_' + str(ano) + '.zip' 
    
    # define nome do arquivo dentro do arquivo zip, a depender do ano
    if ano > 2012:
        arq = 'consulta_cand_' + str(ano) + '_' + uf + '.csv'
        cols = None
    else:
        arq = 'consulta_cand_' + str(ano) + '_' + uf + '.txt'
        if ano == 2012:
            cols = c_12
        else: 
            cols = c_10
    
    # abre arquivo zipado
    zf = ZipFile(nome_zip)
    
    # faz a leitura do arquivo da base de candidatos, usando read_csv
    df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', names=cols, dtype= {'NR_CPF_CANDIDATO': 'string'})
    df['DS_ELEICAO'] = df['DS_ELEICAO'].str.title()
    return df    

In [19]:
# lê dados de candidatos das eleições 2008
cd0 = le_cand(2012,'SP')

In [20]:
# municipios com maior quantidade de candidatos
# o único que atende, individualmente, ao requisito mínimo da Puc-Minas (de 1000 observação) é São Paulo
cd0.groupby('SG_UE').agg({'ANO_ELEICAO':'count'}).sort_values('ANO_ELEICAO', ascending=False).head()

,ANO_ELEICAO
SG_UE,
71072,1255
64777,1146
62910,761
70750,645
71510,568


In [21]:
# filtra apenas candidatos do municipio de São Paulo
cd = cd0.query('SG_UE == 71072')

In [22]:
cd.shape

(1255, 44)

In [23]:
cols = ['CD_CARGO','DS_CARGO']
cd[cols].drop_duplicates()

,CD_CARGO,DS_CARGO
206,12,VICE-PREFEITO
14309,13,VEREADOR
28872,11,PREFEITO


In [24]:
# deixa na base dados inicial somente os dados de candidaturas a vereador 
cd08 = cd.query('CD_CARGO == 13').copy()

In [25]:
# tamanho inicial da base de candidatos
cd08.shape

(1227, 44)

In [26]:
# exclui coluna e-mail, inexistente na base 2008
cd08.drop(columns='NM_EMAIL', inplace=True)

In [27]:
# procura por missing values no dataframe
cd08.isnull().values.any()

False

In [28]:
# procura por registros duplicadoss
cd08[cd08.duplicated()]

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_LEGENDA,SG_LEGENDA,COMPOSICAO_LEGENDA,NM_LEGENDA,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_ELEICAO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO


In [29]:
cd08.sample(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_LEGENDA,SG_LEGENDA,COMPOSICAO_LEGENDA,NM_LEGENDA,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_ELEICAO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
77221,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,KARL HEINZ HADZIC JUNIOR,250000064108,16010,26788645884,KARL HADZIC,6,RENÚNCIA,16,PSTU,PARTIDO SOCIALISTA DOS TRABALHADORES UNIFICADO,250000004062,#NE#,PSTU,PARTIDO ISOLADO,297,SERVIDOR PÚBLICO ESTADUAL,07/06/1979,276422620175,-1,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,SP,-1,SAO PAULO,"200,000.00",-1,#NULO#
70303,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,PAULO EDUARDO BRAGA LIBONATI,250000100990,43007,31622796829,PAULINHO LIBONATI,2,DEFERIDO,43,PV,PARTIDO VERDE,250000000274,#NE#,PV,PARTIDO ISOLADO,257,EMPRESÁRIO,16/07/1983,310897730141,-1,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,SP,-1,SÃO PAULO,"1,500,000.00",5,SUPLENTE
37738,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,DILZA MIEKO MURAMOTO SHIROMA,250000049353,55655,00756331838,DILZA MURAMOTO,2,DEFERIDO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,250000003163,#NE#,PR / DEM / PSDB / PSD,COLIGAÇÃO PSDB/PSD/PR/DEM,125,ADMINISTRADOR,18/09/1955,153192300159,-1,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,MS,-1,ITAPORÃ,"5,000,000.00",5,SUPLENTE


#### Análise incial dos dados de candidatos

In [13]:
# gera arquivos html com análise exploratória inicial dos dados, usando biblioteca sweetviz
testeviz = sv.analyze(cd08)
testeviz.show_html('dfcd08.html')


Report dfcd08.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


### 3.1.1. Observações acerca da análise preliminar dos dados:

#### Colunas sem dados informados, ou com dados imprestáveis:
    - SG_LEGENDA apenas o valor '#NE#' informado para todos os candidatos,
    - COMPOSICAO_LEGENDA apenas o valor '#NE#' infoprmado para todos os candidatos,
    - SQ_LEGENDA não confiável, como será visto a seguir
    - IDADE_DATA_ELEICAO apenas o valor '-1' informado para todos os candidatos, 
    - CODIGO_MUNICIPIO_NASCIMENTO apenas o valor '-1' infoprmado para todos os candidatos, 
    - VR_DESPESA_MAX_CAMPANHA apenas o valor '-1' infoprmado para todos os candidatos

#### Missing Values
Apesar da análise exploratória indicar a inexistência de missing values, os arquivos de apoio do TSE indicam que:

- campos preenchidos com “#NULO” significam que a informação está em branco no banco de dados;
- correspondente para #NULO nos campos numéricos é “-1”;
- campos preenchidos com “#NE” significam que naquele ano a informação não era registrada em banco de dados pelos sistemas eleitorais;
- correspondente para #NE nos campos numéricos é “-3”
- assim, entendemos que campos preenchidos com estes valores são, na verdade, missing values.

In [30]:
# colunas que contem um único valor em todos os registro e respectivo valor
for col in cd08.columns:
    if cd08[col].nunique() == 1:
        print(col, cd08[col].unique())

DT_GERACAO ['15/07/2016']
HH_GERACAO ['19:25:35']
ANO_ELEICAO [2012]
NR_TURNO [1]
DS_ELEICAO ['Eleição Municipal 2012']
SG_UF ['SP']
SG_UE [71072]
NM_UE ['SÃO PAULO']
CD_CARGO [13]
DS_CARGO ['VEREADOR']
SG_LEGENDA ['#NE#']
IDADE_DATA_ELEICAO [-1]
CD_MUNICIPIO_NASCIMENTO [-1]


### 3.1.2. Tratamento missing values:

- SG_LEGENDA e COMPOSICAO_LEGENDA
    - existe fonte de dados alternativa para coletar estas informações
    - excluir colunas relacionadas a coligações e legendas
    
- VR_DESPESA_MAX_CAMPANHA
    - apenas o valor '-1' informado para todos os candidatos
    - não temos fonte alternativa para corrigir o problema
    - excluir coluna pois não terá nenhuma utilidade nas análises

- IDADE_DATA_ELEICAO 
    - apenas o valor '-1' informado para todos os candidatos
    - calcular idade do candidato com base na data de nascimento, disponivel em coluna própria
    - usar "idade na data da posse", ao invés de "idade na data eleição" para fins de padronização

- CODIGO_MUNICIPIO_NASCIMENTO 
    - apenas o valor '-1' infoprmado para todos os candidatos: imprestável
    - tentar recuperar nome e codigo de municipio de nascimento de tabela do IBGE

#### Exclui colunas imprestáveis da base de candidatos

#### 3.1.2.1 Exclui colunas relacionadas a base de coligações

In [31]:
# exclui colunas inconsistentes da base candidatos relacionadas a coligações:
remove_cols = ['SQ_LEGENDA','SG_LEGENDA','COMPOSICAO_LEGENDA','NM_LEGENDA' ]
cd08.drop(remove_cols, axis=1, inplace=True)

#### 3.1.2.2 Exclui outras colunas imprestáveis

In [32]:
# exclui colunas NaN da base candidatos:
remove_cols = ['VR_DESPESA_MAX_CAMPANHA' ]
cd08.drop(remove_cols, axis=1, inplace=True)

#### 3.1.2.3 Calcula idade na data da posse e coloca na coluna IDADE_DATA_ELEICAO
 - na base de candidatos do TSE para o ano de 2008 este campo veio todo com valor '-1', ou seja, NaN
 - temos o campo data de nascimento
 - vamos calcular a idade do candidato com base em sua data de nascimento
 - a partir do ano 2014, a base do TSE passa trazer o campo idade na data da posse.
 - para manter padronização, nosso novo campo calculado será idade na data da posse

In [33]:
# altera tipo do campo DT_NASCIMENTO para daytime
from datetime import date, timedelta
cd08['DT_NASCIMENTO'] = pd.to_datetime(cd08['DT_NASCIMENTO'], dayfirst=True, errors='coerce')

In [34]:
# calcula idade na data da posse (a posse é sempre no dia 1 de janeiro do ano seguinte ao das eleições)
cd08['IDADE_DATA_ELEICAO'] = (pd.to_datetime('2009-01-01') - cd08['DT_NASCIMENTO'])//timedelta(days=365.2425)

In [35]:
# renomerar coluna para idade_data_posse
cd08.rename(columns={'IDADE_DATA_ELEICAO':'IDADE_DATA_POSSE'}, inplace=True)

In [37]:
# algumas datas de nascimento tem valores inválidos para data, o que gerou NaN na conversão
# ajusta NaN e corrige tipo da coluna
cd08['IDADE_DATA_POSSE'].fillna(-1,inplace=True)
cd08['DT_NASCIMENTO'].fillna(-1,inplace=True)
cd08['IDADE_DATA_POSSE'] = cd08['IDADE_DATA_POSSE'].astype(int)

In [38]:
cd08.sample(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO
14371,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,GABRIEL MEDINA DE TOLEDO,250000081997,13321,22379334803,GABRIEL MEDINA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,132,PSICÓLOGO,1982-01-16 00:00:00,297881190116,26,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,SP,-1,ARARAQUARA,5,SUPLENTE
67057,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,ANDERSON OLIVEIRA DE JESUS,250000007961,36023,27653916880,ANDERSON OLIVEIRA,2,DEFERIDO,36,PTC,PARTIDO TRABALHISTA CRISTÃO,301,DIRETOR DE EMPRESAS,1980-05-23 00:00:00,275364540124,28,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,SP,-1,SAO PAULO,5,SUPLENTE
14317,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,ADIR GOMES TEIXEIRA,250000081970,13010,01193081858,ADIR,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1956-07-04 00:00:00,77738890167,52,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,MG,-1,SÃO MIGUEL DO ANTA,5,SUPLENTE


#### 3.1.2.4 Tentativa recuperar codigo municipio nascimento da base de municipios TSE
- na base de dados de candidatos, a coluna cd_municipio_nascimento está sem valores informados (-1) = NaN
- por outro lado a coluna nm_municio_nascimento tem valores preenchidos
- entretanto é uma coluna de livre preenchimento e sem criticas, portanto sujeita a todo tipo de erros
- vamos tentar recuperar o código do município do TSE a partir da UF e do nome do informados pelo candidato (estão na tabela candidatos), comparando com o nome dos municipios constantes da tabela do TSE

#### Estratégia usada para tentar encontrar o nome correto do município e seu respectivo código:
    1) Preparação das bases de dados
        
        1.1) da nossa base de canditados, extraímos uma base mais enxuta, apenas com os campos:
            - SQ_CANDIDATO
            - UF
            - NM_MUNICIPIO_NASCIMENTO
        
        1.2) bases de referência para comparação:
            - o TSE usa codificação de municípios própria, porém não tem informação de distritos/subdistritos. 
            - O IBGE tem codificação detalhada para cada, município, distrito e subdistrito, dentre outros
            - juntamos a codificação do IBGE até o nível de subdistritos e acrescentamos a ela o codigo de municipio usado pelo TSE. 
            - esta foi a nossa base final, que serviu de referencia para as pesquisas
            
    2) Normalização das bases 
        - Como a coluna NM_MUNICIPO_NASCIMENTO da base de canditados do TSE é de livre preenchimento e, portanto, sujeita a todo tipo de erros, inicialmente tentamos "normalizar" esta coluna para facilitar na etapa de pesquisas. a pesquisa. Assim, o primeiro passo foi:
        
        2.1) deixar todos os nomes de municipios em minúsculas (tanto base IBGE/TSE quanto na base candidatos)
        2.2) retirar a acentuação (na base IBGE/TSE e na base candidatos)
    
    3) Feita a normalização inicial, executamos comparações de semelhança entre entre os nomes informados na base de candidatos e os nomes constantes da base do IBGE/TSE, na seguinte ordem:
    
    3.1) busca se o par (uf, nome_município) informados na base candidatos tem correlação IDÊNTICA na base do IBGE/TSE:
        - para as linhas que tiveram correlação idêndica, recuperamos da base do IBGE/TSE o código de município do TSE e separamos em uma base de sucesso, que posteriormente atualizará a base de candidatos
        - os registros que não tiveram correlação positiva foram separados em um novo dataframe, para tentar tratar na etapa seguinte.
    
    3.2) busca se o par (uf, nome_município) informados na base candidatos tem correlação IDÊNTICA no par (uf, nome_distrito) da base do IBGE
        - linhas com correlação idêntica, recuperamos o código de município do TSE
        - linhas sem correlação, separadas para a proxima etapa de busca

    3.3) busca se o par (uf, nome_município) informados na base candidatos tem correlação IDÊNTICA no par (uf, nome_subdistrito) da base do IBGE
        - linhas com correlação idêntica, recuperamos o código de município do TSE
        - linhas sem correlação, separadas para a proxima etapa de busca

    3.4) busca se o par (uf, nome_município) informados na base candidatos tem correlação SIMILAR no par (uf, nome_município) da base do IBGE
        - linhas com correlação de semelhança, recuperamos o código de município do TSE
        - linhas sem correlação, separadas para a proxima etapa de busca
            
    3.5) busca se o par (uf, nome_município) informados na base candidatos tem correlação SIMILAR no par (uf, nome_distrito) da base do IBGE
        - linhas com correlação de semelhança, recuperamos o código de município do TSE
        - linhas sem correlação, separadas para a proxima etapa de busca

    3.6) busca se o par (uf, nome_município) informados na base candidatos tem correlação SIMILAR no par (uf, nome_subdistrito) da base do IBGE
        - linhas com correlação de semelhança, recuperamos o código de município do TSE
        - linhas sem correlação, separadas para a proxima etapa de busca

    3.6) tratar por códigos de UF extrangeiros
        - conforme descrito no 'leia.me' dos arquivos do TSE, a campo UF pode conter, além da unidade da federação, o valor valor 'ZZ' quando se tratar de localidade no exterior
        - para estes casos, nos pareceu válido usar um código específico para candidatos nascidos fora do Brasil
        - imputamos o valor -5 para estes casos
    
    3.7) busca se o (nome_município) informados na base candidatos tem correlação SIMILAR em (nome_município, nome_distrito e nome_subdistrito) da base do IBGE
        - linhas com correlação de semelhança, recuperamos o código de município do TSE
        - linhas sem correlação, não serão mais tratadas e reberão valor -1 

*  para realizar as pesquisas de similaridade utilizamos a biblioteca difflib do python. Esta biblioteca é baseada no algoritmo publicado por de Ratcliff and Obershelp, no final dos anos 80, com o nome 'gestalt pattern matching'.
* está biblioteca tem um parametro de "corte": cutoff:
    - cutoff varia de 0 a 1, 
    - o valor 1 indica pesquisa IDÊNTICA
    - quanto mais proximo de 0 mais tolerável a erro
    - o padrão da biblioteca é 0.6
    - após algumas observações, decidimos usar o valor 0.65, por entender ser um valor  adequado ao caso

* tabela de distritos obtida portal ibge:
* https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html?=&t=downloads

* tabela de municípios TSE, baixada de:
* https://github.com/betafcc/Municipios-Brasileiros-TSE


#### Dataset resumido de candidatos

In [39]:
# cria dataset resumido com com cod candidato e nome municipio nascimento
cols = ['SQ_CANDIDATO','SG_UF_NASCIMENTO','NM_MUNICIPIO_NASCIMENTO']
uen = cd08[cols].copy()
# renomeia coluna uf para ficar no mesmo padrão da tabela tse
uen.rename(columns={'SG_UF_NASCIMENTO':'uf', 'NM_MUNICIPIO_NASCIMENTO':'nm_mun_dig'}, inplace=True)
# normaliza nome do municipio - cria coluna com o nome em caixa baixa e sem acentos
uen['muni_norm'] = uen['nm_mun_dig'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1').str.lower()

In [40]:
uen.head(3)

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
14309,250000082035,SP,SÃO PAULO,sao paulo
14310,250000082043,BA,IBIRATAIA,ibirataia
14311,250000082076,SP,EMILIANOPOLIS,emilianopolis


#### Montagem da base de pesquisa / comparação

In [41]:
from unicodedata import normalize #importa biblioteca para retirar acentuação de palavras
import difflib #importa biblioteca difflib para pesquisas de palavras por semelhança

#### Importa tabela com codigos do IBGE de 2008

In [42]:
# lendo tabelas de distritos e subdistritos IBGE de 2008
zf = ZipFile('ibge/dtb_2008.zip')
arq = 'DTB_2008.xls'
tipos = {'UF':'string','Município':'string'}
# lista das colunas de ineteresse da tabela do IBGE
cols = ['UF','Município','Nome_Município','Nome_Distrito','Nome_SubDistrito']
# lê tabela xls do IBGE armazena resultado em dataframe pandas, apenas com as colunas de interesse
dist_ibge = pd.read_excel(zf.open(arq), dtype=tipos)[cols]
# cria coluna com código completo do município (cod uf + cod municipio)
#dist_ibge['cod_mun'] = dist_ibge['UF'] + dist_ibge['Município']

In [43]:
dist_ibge.head(2)

,UF,Município,Nome_Município,Nome_Distrito,Nome_SubDistrito
0,11,00015,Alta Floresta D'Oeste,Rolim de Moura do Guaporé,NaN
1,11,00015,Alta Floresta D'Oeste,Izidolândia,NaN


In [44]:
# dicionário com siglas e códigos das UF
dict_uf = {'11':'RO', '12':'AC', '13':'AM', '14':'RR', '15':'PA',
           '16':'AP', '17':'TO', '21':'MA', '22':'PI', '23':'CE',
           '24':'RN', '25':'PB', '26':'PE', '27':'AL', '28':'SE',
           '29':'BA', '31':'MG', '32':'ES', '33':'RJ', '35':'SP',
           '41':'PR', '42':'SC', '43':'RS', '50':'MS', '51':'MT',
           '52':'GO', '53':'DF'}

In [45]:
# cria coluna com sigla do uf
dist_ibge['uf'] = dist_ibge['UF'].map(dict_uf)
# cria coluna com codigo de município do IBGE completo 
dist_ibge['cd_mun_ibge'] = (dist_ibge['UF'] + dist_ibge['Município']).astype(int)
# lista das colunas de ineteresse
cols_final = ['uf','cd_mun_ibge','Nome_Município','Nome_Distrito','Nome_SubDistrito']
# cria df ibge apenas com colunas de ineteresse
tab_ibge = dist_ibge[cols_final]
# renomeia colunas do df
tab_ibge.columns = ['uf','cd_mun_ibge','nm_mun','nm_dist','nm_subdist']

In [46]:
tab_ibge.head(2)

,uf,cd_mun_ibge,nm_mun,nm_dist,nm_subdist
0,RO,1100015,Alta Floresta D'Oeste,Rolim de Moura do Guaporé,NaN
1,RO,1100015,Alta Floresta D'Oeste,Izidolândia,NaN


In [47]:
tab_ibge.shape

(10575, 5)

In [48]:
# cria tabelas individualizadas para municipios, distritos e subdistritos
muni = ['uf','cd_mun_ibge','nm_mun']
dist = ['uf','cd_mun_ibge','nm_dist']
subd = ['uf','cd_mun_ibge','nm_subdist']
tb_muni = tab_ibge[muni].copy().drop_duplicates().sort_values(['uf','cd_mun_ibge'])
tb_dist = tab_ibge[dist].copy().drop_duplicates().dropna(subset=['nm_dist']).sort_values(['uf','cd_mun_ibge'])
tb_subd = tab_ibge[subd].copy().drop_duplicates().dropna(subset=['nm_subdist']).sort_values(['uf','cd_mun_ibge'])

tb_muni['nm_mun'] = tb_muni['nm_mun'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1').str.lower()
tb_dist['nm_dist'] = tb_dist['nm_dist'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1').str.lower()
tb_subd['nm_subdist'] = tb_subd['nm_subdist'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1').str.lower()

In [49]:
tb_muni.head(2)

,uf,cd_mun_ibge,nm_mun
101,AC,1200013,acrelandia
102,AC,1200054,assis brasil


In [50]:
tb_muni.shape

(5564, 3)

In [51]:
tb_dist.head(2)

,uf,cd_mun_ibge,nm_dist
101,AC,1200013,acrelandia
102,AC,1200054,assis brasil


In [52]:
tb_dist.shape

(10104, 3)

In [53]:
tb_subd.head(2)

,uf,cd_mun_ibge,nm_subdist
2876,AL,2704302,primeira regiao
2877,AL,2704302,quarta regiao


In [54]:
tb_subd.shape

(460, 3)

#### Importa tabela com códigos do TSE

In [55]:
# le tabela de municipios tse/ibge
muni_tse = pd.read_csv('tse/municipios_brasileiros_tse_2.csv')
# cria coluna com nome normalizado (sem acentos e tudo minuscula)
muni_tse['muni_norm'] = muni_tse['nome_municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1').str.lower()
# renomeia colunas 
muni_tse.columns = ['cd_tse','uf', 'nm_municipio', 'capital', 'cd_mun_ibge', 'muni_norm']
# colunas de interesse
cols_fim = ['uf', 'cd_tse', 'cd_mun_ibge', 'muni_norm']
tb_tse = muni_tse[cols_fim].copy()
tb_tse.rename(columns={'muni_norm':'nm_mun'}, inplace=True)

In [56]:
muni_tse.head(2)

,cd_tse,uf,nm_municipio,capital,cd_mun_ibge,muni_norm
0,1120,AC,ACRELÂNDIA,0,1200013,acrelandia
1,1570,AC,ASSIS BRASIL,0,1200054,assis brasil


In [57]:
tb_tse.head(2)

,uf,cd_tse,cd_mun_ibge,nm_mun
0,AC,1120,1200013,acrelandia
1,AC,1570,1200054,assis brasil


In [58]:
# acrescenta a tabela de municipios do ibge o cod municipio e nome municipio, conforme tabela tse
cols = ['cd_tse','cd_mun_ibge','nm_mun']
tb_muni = tb_muni.merge(tb_tse[cols], how='left', on=['cd_mun_ibge'], suffixes=(None, '_tse'))

In [59]:
tb_muni.head(2)

,uf,cd_mun_ibge,nm_mun,cd_tse,nm_mun_tse
0,AC,1200013,acrelandia,1120,acrelandia
1,AC,1200054,assis brasil,1570,assis brasil


In [60]:
tb_muni.shape

(5564, 5)

In [61]:
# acrescenta a tabela de distritos do ibge o cod municipio, conforme tabela tse
cols = ['cd_tse','cd_mun_ibge']
tb_dist = tb_dist.merge(tb_tse[cols], how='left', on=['cd_mun_ibge'], suffixes=(None, '_tse'))

In [62]:
tb_dist.head(2)

,uf,cd_mun_ibge,nm_dist,cd_tse
0,AC,1200013,acrelandia,1120
1,AC,1200054,assis brasil,1570


In [63]:
tb_dist.shape

(10104, 4)

In [64]:
# acrescenta a tabela de municipios do ibge o cod municipio e nome municipio, conforme tabela tse
cols = ['cd_tse','cd_mun_ibge']
tb_subd = tb_subd.merge(tb_tse[cols], how='left', on=['cd_mun_ibge'], suffixes=(None, '_tse'))

In [65]:
tb_subd.head(2)

,uf,cd_mun_ibge,nm_subdist,cd_tse
0,AL,2704302,primeira regiao,27855
1,AL,2704302,quarta regiao,27855


In [66]:
tb_subd.shape

(460, 4)

#### Função para fazer as pesquisas e retornar os dataframes

In [68]:
# com opção de não forçar buscas dentro do estado informado pelo candidato
import difflib
def busca_semelhanca(df_base,tipo,corte,filtro):
    df = pd.DataFrame()
    col_resultado = 'diff_' + tipo + '_corte_' + str(corte) + '_' + filtro
    # define tabela (df) e campo (coluna) onde buscar 
    if tipo == 'm':
        tab_base = tb_tse
        col_pesquisa = 'nm_mun'
        cols_tse=['cd_tse', 'uf', 'nm_mun']
    elif tipo == 'm1':
        tab_base = tb_muni
        col_pesquisa = 'nm_mun_tse'
        cols_tse=['cd_tse', 'uf', 'nm_mun_tse']
    elif tipo == 'm2':
        tab_base = tb_muni
        col_pesquisa = 'nm_mun'
        cols_tse=['cd_tse', 'uf', 'nm_mun']
    elif tipo == 'd':
        tab_base = tb_dist
        col_pesquisa = 'nm_dist'
        cols_tse=['cd_tse', 'uf', 'nm_dist']
    elif tipo == 's':
        tab_base = tb_subd
        col_pesquisa = 'nm_subdist'
        cols_tse=['cd_tse', 'uf', 'nm_subdist']
    
    if filtro == 'uf': # com filtro em uf
        # percorre 
        for estado in df_base['uf'].unique():
            # cria tabela tse/ibge somente para um estado especifico do 'for'
            dist_f = tab_base[tab_base['uf'] == estado]
            # cria df com uf nascimento somente de um estado especifo do 'for'
            uen_f = df_base[df_base['uf'] == estado].copy()
            f = lambda x: next(iter(difflib.get_close_matches(x, dist_f[col_pesquisa], cutoff=corte, n=1)), np.NaN)
            uen_f[col_resultado] = (uen_f['muni_norm']).apply(f)
            df = df.append(uen_f)
        df[col_resultado] = df[col_resultado].astype(str)
        # agrega o código tse (do df muni_tse) as linhas que obtiveram resultado na busca
        df = df.merge(tab_base[cols_tse], how='left', left_on=['uf',col_resultado], right_on=['uf',col_pesquisa])

    elif filtro == 'br': # sem filtro em uf
        df = df_base.copy()
        f = lambda x: next(iter(difflib.get_close_matches(x, tab_base[col_pesquisa], cutoff=corte, n=1)), np.NaN)
        df[col_resultado] = (df['muni_norm']).apply(f)
        df[col_resultado] = df[col_resultado].astype(str)
        # agrega o código tse (do df tb_tse) as linhas que obtiveram resultado na busca
        df = df.merge(tab_base[cols_tse], how='left', left_on=[col_resultado], right_on=[col_pesquisa])
        df.drop(columns=['uf_y','nm_mun'], inplace=True)
        df.rename(columns={'uf_x':'uf'}, inplace=True)
        
    # separa o resultado em 2 df: 1 que achou municipio e outro que não encontrou
    df.rename(columns={'muni_norm_x':'muni_norm'}, inplace=True)
    df1 = df[df.cd_tse.notna()].copy()
    cols1 = [0,1,2,4,5]
    df1 = df1[df1.columns[cols1]]
    cols = ['SQ_CANDIDATO','uf','nm_mun_dig','muni_norm']
    df2 = df[df.cd_tse.isna()].copy()[cols]
    print(f'{df_base.shape[0]} municipios pesquisados')
    print(f'{round(df1.shape[0] / df_base.shape[0] *100,2)}% ({df1.shape[0]}) encontrados')
    print(f'{round(df2.shape[0] / df_base.shape[0] *100,2)}% ({df2.shape[0]}) não encontrados')
    return df1,df2

#### 1a pesquisa para recuperar nome e codigo de municipios
- compara o par uf e nome do municipio informados pelo candidato
- retorna codigos apenas se encontar o par exato "uf" e "nome municipio" na base do IBGE/TSE
- pode ser feita via merge direto ou usando difflib com cutoff 1 (mesmo resultado)

In [69]:
# 1a busca: nome do par (uf, municipio) identico, entre o digitado pelo candidato e o nome TSE
tok,tbad = busca_semelhanca(uen, 'm2', 1, 'uf')

1227 municipios pesquisados
94.38% (1158) encontrados
5.62% (69) não encontrados


In [70]:
tok.head()

,SQ_CANDIDATO,uf,nm_mun_dig,diff_m2_corte_1_uf,cd_tse
0,250000082035,SP,SÃO PAULO,sao paulo,"71,072.00"
1,250000082076,SP,EMILIANOPOLIS,emilianopolis,"61,387.00"
2,250000082030,SP,NIPOÃ,nipoa,"67,555.00"
3,250000082074,SP,SÃO PAULO,sao paulo,"71,072.00"
4,250000082060,SP,SÃO PAULO,sao paulo,"71,072.00"


In [71]:
tbad.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
136,250000080663,SP,MIRANTE PARANAPANEMA,mirante paranapanema
154,250000049427,SP,MATINÓPOLIS,matinopolis
191,250000049396,SP,PARAGAÇU PAULISTA,paragacu paulista
246,250000039137,SP,VELACIO DE ALAGOAS,velacio de alagoas
260,250000039094,SP,NOSSA SENHORA DAS DORES,nossa senhora das dores


In [72]:
# 1a busca (complemento): nome do par (uf, municipio) identico, entre o digitado pelo candidato e nome IBGE
tok1,tbad1 = busca_semelhanca(tbad, 'm', 1, 'uf')

69 municipios pesquisados
0.0% (0) encontrados
100.0% (69) não encontrados


#### A grande maioria dos códigos de municipios foram recuperados com a primeira tentativa
- desconsiderando acentos e deixando tudo em caixa baixa. 
- recuperado 59568 do total de 63605 (93,65%)

#### Possivelmente existem outras inconsistencias nos cerca de 4000 não recuperados
- como se trata de um campo de livre preenchimento, seria normal uma quantidade grande de erros
- vamos tentar recuperar por semelhança entre as colunas uf e nome municipio 

#### 2a Pesquisa para recuperar nome e codigo de municipios
- compara o par uf e nome do municipio informados pelo candidato
- retorna codigos apenas se encontar o par exato em "uf" e "nome do distrito" na base do IBGE/TSE

In [73]:
# 2a busca: nome do par (uf, municipio) digitado identico ao par (uf, subdistrito) da base ibge/tse
tok2,tbad2 = busca_semelhanca(tbad1, 's', 1, 'uf')

69 municipios pesquisados
1.45% (1) encontrados
98.55% (68) não encontrados


In [74]:
tok2.head(3)

,SQ_CANDIDATO,uf,nm_mun_dig,diff_s_corte_1_uf,cd_tse
64,250000010994,DF,PLANALTINA,planaltina,"97,012.00"


In [75]:
tbad2.head(3)

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
0,250000080663,SP,MIRANTE PARANAPANEMA,mirante paranapanema
1,250000049427,SP,MATINÓPOLIS,matinopolis
2,250000049396,SP,PARAGAÇU PAULISTA,paragacu paulista


#### 3a Pesquisa para recuperar nome e codigo de municipios
- compara o par uf e nome do municipio informados pelo candidato
- retorna codigos apenas se encontar o par exato em "uf" e "nome do distrito" na base do IBGE/TSE

In [76]:
# 3a busca: nome do par (uf, municipio) digitado, identico a (uf, distrito) na base ibge/tse
tok3,tbad3 = busca_semelhanca(tbad2, 'd', 1, 'uf')

68 municipios pesquisados
2.94% (2) encontrados
97.06% (66) não encontrados


In [77]:
tok3.head(3)

,SQ_CANDIDATO,uf,nm_mun_dig,diff_d_corte_1_uf,cd_tse
10,250000010998,SP,CORONEL GOULART,coronel goulart,"61,255.00"
34,250000019012,PE,JABOATÃO,jaboatao,"24,570.00"


In [78]:
tbad3.sample(3)

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
23,250000082016,PR,CARAMBÁ,caramba
62,250000007558,ZZ,URUGUAI,uruguai
25,250000002223,PR,PARANA,parana


#### Percebe-se que a junção aumentou o numero de linhas na tabela uen_2
- tal fato ocorre devido a existencia de nomes de distritos identicos em um mesmo estado, porem em municipios diferentes.
- como tratar ??? 
- escolheremos o primeiro encontrado e descartaremos os demais

In [79]:
# lista dos duplicados
tok3[tok3.duplicated(subset=['SQ_CANDIDATO','uf','diff_d_corte_1_uf'])]

,SQ_CANDIDATO,uf,nm_mun_dig,diff_d_corte_1_uf,cd_tse


#### Ainda temos 3264 candidatos com municipios não encontrados depois das 3 tentativas iniciais
- vamos tentar uma busca por proximidade no nome municipio

In [80]:
# 4a busca: nome de uf identica e municipio similar (digitado vs base ibge/tse)
tok4,tbad4 = busca_semelhanca(tbad3, 'm2', 0.7, 'uf')

66 municipios pesquisados
60.61% (40) encontrados
39.39% (26) não encontrados


In [81]:
tok4.sample(10)

,SQ_CANDIDATO,uf,nm_mun_dig,diff_m2_corte_0.7_uf,cd_tse
20,250000007956,BA,RUI BARBOSA,ruy barbosa,"38,458.00"
12,250000008004,SP,BANANEIRAS,cananeia,"62,995.00"
37,250000008021,RJ,ITAGUI,itaguai,"58,394.00"
16,250000018945,SP,ASSAI,assis,"61,794.00"
46,250000018986,MA,GOVERNADOR ARCHE,governador archer,"7,870.00"
5,250000039120,SP,BRASILEIRA,silveiras,"71,412.00"
33,250000018978,PE,CUSTODIO,custodia,"24,015.00"
26,250000039106,MG,MINAS GERAIS,campos gerais,"42,315.00"
30,250000007958,AL,S. M. DOS CAMPOS,sao miguel dos campos,"28,711.00"
27,250000002331,MG,SANTA CRUZ DO ESCOLVADO,santa cruz do escalvado,"51,470.00"


In [82]:
tbad4.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
3,250000039137,SP,VELACIO DE ALAGOAS,velacio de alagoas
4,250000039094,SP,NOSSA SENHORA DAS DORES,nossa senhora das dores
7,250000039124,SP,MOGADOURO,mogadouro
8,250000039105,SP,RAUL SOARES - MINAS GERAIS,raul soares - minas gerais
10,250000008012,SP,SACRAMENTO,sacramento


#### Resultados da busca por proximidade no nome do municipio, obedecendo o estado
- Esta busca por proximidade com offset de 0.65 recuperou 81,8% dos nomes que ainda faltavam
- Aparentemente este foi um offset bom para o nosso problema
- faremos a mesma busca por proximidade em distrito e subdistrito a seguir

In [83]:
# 5a busca: nome de uf identica e subdistrito similar (digitado vs base ibge/tse)
tok5,tbad5 = busca_semelhanca(tbad4, 's', 0.8, 'uf')

26 municipios pesquisados
0.0% (0) encontrados
100.0% (26) não encontrados


In [84]:
tok5.head()

,SQ_CANDIDATO,uf,nm_mun_dig,diff_s_corte_0.8_uf,cd_tse


In [85]:
tbad5.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
0,250000039137,SP,VELACIO DE ALAGOAS,velacio de alagoas
1,250000039094,SP,NOSSA SENHORA DAS DORES,nossa senhora das dores
2,250000039124,SP,MOGADOURO,mogadouro
3,250000039105,SP,RAUL SOARES - MINAS GERAIS,raul soares - minas gerais
4,250000008012,SP,SACRAMENTO,sacramento


In [86]:
# 6a busca: nome de uf identica e distrito similar (digitado vs base ibge/tse)
tok6,tbad6 = busca_semelhanca(tbad5, 'd', 0.8, 'uf')

26 municipios pesquisados
0.0% (0) encontrados
100.0% (26) não encontrados


In [87]:
tok6.head()

,SQ_CANDIDATO,uf,nm_mun_dig,diff_d_corte_0.8_uf,cd_tse


In [88]:
tbad6.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
0,250000039137,SP,VELACIO DE ALAGOAS,velacio de alagoas
1,250000039094,SP,NOSSA SENHORA DAS DORES,nossa senhora das dores
2,250000039124,SP,MOGADOURO,mogadouro
3,250000039105,SP,RAUL SOARES - MINAS GERAIS,raul soares - minas gerais
4,250000008012,SP,SACRAMENTO,sacramento


#### Feitas as buscas por semelhança entre o nome de municipio digitado, comparado com municipios, distritos e subdistritos:
    - encontramos a grande maioria dos 3461 nomes
    - sobraram ainda 593 nomes não identificados


#### Proximos passos:
    - a UF ZZ na base do TSE indica exterior
    - marcar como codigo_tse "-5" os registros com UF == ZZ (exterior)
    - fazer ultima busca por proximidade, sem trava de estado, ou seja:
    - buscar em todos os municipios do Brasil, pelo nome digitado pelo candidato

In [89]:
# 7a intervenção: trata supostamente nascidos no exterior
tbad6.query('uf=="ZZ"')

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
11,250000081978,ZZ,PORTUGAL,portugal
12,250000082045,ZZ,JAPÃO,japao
13,250000080653,ZZ,TAIWAN,taiwan
14,250000049431,ZZ,COREIA,coreia
15,250000002384,ZZ,CORÉIA DO SUL - SEUL,coreia do sul - seul
16,250000002407,ZZ,TAIWAN CHINA,taiwan china
17,250000002262,ZZ,SANTIAGO,santiago
18,250000002292,ZZ,OKINAWA,okinawa
19,250000002287,ZZ,IRÃ,ira
20,250000007539,ZZ,LIBANO,libano


In [90]:
# cria df somente com uf = ZZ
# indice das linhas que tem uf = ZZ
ind = tbad6.query('uf=="ZZ"').index
# separa estes registros em outro dataframe
tok7 = tbad6.loc[ind].copy()
# imputa valores de cod_tse e coluna nome ao novo df (para manter coerencia com os outros já "encontrados")
tok7['codigo_tse'] = -5
tok7.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm,codigo_tse
11,250000081978,ZZ,PORTUGAL,portugal,-5
12,250000082045,ZZ,JAPÃO,japao,-5
13,250000080653,ZZ,TAIWAN,taiwan,-5
14,250000049431,ZZ,COREIA,coreia,-5
15,250000002384,ZZ,CORÉIA DO SUL - SEUL,coreia do sul - seul,-5


In [91]:
# cria df somente com uf <> ZZ
# indice das linhas que tem uf = ZZ
ind = tbad6.query('uf!="ZZ"').index
# separa estes registros em outro dataframe
tbad7 = tbad6.loc[ind].copy()
tbad7.shape

(15, 4)

In [92]:
# 8a busca: nome de municipio similar sem filtro por uf (digitado vs base ibge/tse)
tok8,tbad8 = busca_semelhanca(tbad7, 'm', 0.7, 'br')

15 municipios pesquisados
100.0% (15) encontrados
13.33% (2) não encontrados


In [93]:
tok8.head(5)

,SQ_CANDIDATO,uf,nm_mun_dig,diff_m_corte_0.7_br,cd_tse
0,250000039137,SP,VELACIO DE ALAGOAS,estrela de alagoas,"27,049.00"
1,250000039094,SP,NOSSA SENHORA DAS DORES,nossa senhora das dores,"31,917.00"
2,250000039124,SP,MOGADOURO,logradouro,"19,160.00"
4,250000008012,SP,SACRAMENTO,sacramento,"51,373.00"
5,250000002370,PR,MONTE ALEGRE,monte alegre,"4,952.00"


In [94]:
tbad8.head(5)

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm
3,250000039105,SP,RAUL SOARES - MINAS GERAIS,raul soares - minas gerais
12,250000051509,RS,GUAPORÉ MUSSUM,guapore mussum


#### Até este ponto, sobraram ainda 152 linhas sem identificação do municipio
    - estes casos poderiam ser tratados de forma individualiza
    - por questões de tempo e objetivos do projeto, preferimos deixa-los com o código "-1" no codigo_tse

In [95]:
# imputa "-1" como codigo para s municipios que sobrarm sem identificação
# imputa valores de cod_tse e coluna nome ao novo df (para manter coerencia com os outros já "encontrados")
tok9 = tbad8.copy()
tok9['codigo_tse'] = -1
tok9.head()

,SQ_CANDIDATO,uf,nm_mun_dig,muni_norm,codigo_tse
3,250000039105,SP,RAUL SOARES - MINAS GERAIS,raul soares - minas gerais,-1
12,250000051509,RS,GUAPORÉ MUSSUM,guapore mussum,-1


#### Resta ainda:
    - juntar as bases com resultado encontrado
    - excluir duplicados definindo qual manter
    - preparar base para atualizar df de candidatos

In [96]:
# dataframes de resultados
dfs = [tok, tok1, tok2, tok3, tok4, tok5, tok6, tok7, tok8, tok9]
dfs1 = [tok, tok1, tok2, tok3, tok4, tok5, tok6, tok8]
dfs2 = [tok7, tok9]
new_cols = ['SQ_CANDIDATO', 'uf', 'NM_MUNICIPIO_NASCIMENTO', 'RESULTADO', 'cd_tse']
for df in dfs:
    df.columns = new_cols
tok_f1 = pd.concat(dfs1, axis=0, ignore_index=True)
tok_f1.drop(columns=['uf', 'NM_MUNICIPIO_NASCIMENTO', 'RESULTADO'], inplace=True)
tok_f1['cd_tse'] = tok_f1['cd_tse'].astype(int)
tok_f2 = pd.concat(dfs2, axis=0, ignore_index=True)
tok_f2.drop(columns=['RESULTADO'], inplace=True)
tok_f2['cd_tse'] = tok_f2['cd_tse'].astype(int)

In [97]:
tok_f1.shape

(1216, 2)

In [98]:
# excluido duplicado, mantendo a primeira ocorrencia
tok_f1.drop_duplicates(subset=['SQ_CANDIDATO'], keep='first', inplace=True)

In [99]:
tok_f1.shape

(1214, 2)

In [100]:
tok_f1.sample(2)

,SQ_CANDIDATO,cd_tse
1201,250000039137,27049
606,250000019080,71072


In [101]:
tok_f2.sample(5)

,SQ_CANDIDATO,uf,NM_MUNICIPIO_NASCIMENTO,cd_tse
1,250000082045,ZZ,JAPÃO,-5
3,250000049431,ZZ,COREIA,-5
5,250000002407,ZZ,TAIWAN CHINA,-5
10,250000007558,ZZ,URUGUAI,-5
8,250000002287,ZZ,IRÃ,-5


In [102]:
# recupera nome no município e uf para f1
muni_cols = ['cd_tse', 'uf', 'nm_municipio']
tok_f1 = tok_f1.merge(muni_tse[muni_cols], how='left', on=['cd_tse'])
col_order = ['SQ_CANDIDATO', 'uf', 'nm_municipio', 'cd_tse']
tok_f1 = tok_f1[col_order]
tok_f1.rename(columns={'nm_municipio':'NM_MUNICIPIO_NASCIMENTO'}, inplace=True)

In [103]:
tok_f1

,SQ_CANDIDATO,uf,NM_MUNICIPIO_NASCIMENTO,cd_tse
0,250000082035,SP,SÃO PAULO,71072
1,250000082076,SP,EMILIANÓPOLIS,61387
2,250000082030,SP,NIPOÃ,67555
3,250000082074,SP,SÃO PAULO,71072
4,250000082060,SP,SÃO PAULO,71072
...,...,...,...,...
1209,250000011020,BA,MORRO DO CHAPÉU,37354
1210,250000039102,PE,OURICURI,24970
1211,250000049408,MS,TRÊS LAGOAS,91650
1212,250000017666,MS,DOURADOS,90735


In [104]:
# junta as bases tok final
tok_fim = pd.concat([tok_f1, tok_f2], axis=0, ignore_index=True)

In [105]:
tok_fim.rename(columns={'cd_tse':'CD_MUNICIPIO_NASCIMENTO',
                        'uf':'SG_UF_NASCIMENTO'}, inplace=True)

In [106]:
# compara tamanho das bases inicial e final
print(f'{uen.shape[0]} registros na base incial')
print(f'{tok_fim.shape[0]} registros na base final')                   

1227 registros na base incial
1227 registros na base final


#### passo final: atualizar a base de candidatos com os valores de tok_fim

In [107]:
tok_fim.sample(2)

,SQ_CANDIDATO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO
715,250000017644,SP,SÃO PAULO,71072
275,250000039059,SP,SÃO PAULO,71072


In [108]:
# junta as novas colunas a base de candidatos, renomeia as antigas com o sufixo _old
cd08 = cd08.merge(tok_fim, how='left', on=['SQ_CANDIDATO'], suffixes=('_old', ''))

In [109]:
# remove colunas antigas 
cd08.drop(columns=['SG_UF_NASCIMENTO_old', 
                   'CD_MUNICIPIO_NASCIMENTO_old', 
                   'NM_MUNICIPIO_NASCIMENTO_old'], inplace=True)

In [110]:
cd08.sample(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO
312,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,JOSÉ CARLOS OLIVEIRA,250000049383,55055,07419581800,OLIVEIRA,2,DEFERIDO,55,PSD,PARTIDO SOCIAL DEMOCRÁTICO,999,OUTROS,1965-10-25 00:00:00,77515350183,43,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SE,MACAMBIRA,31739
348,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,SERGIO SIQUEIRA DA CRUZ,250000051524,54321,46641971791,DR. SÉRGIO CRUZ,2,DEFERIDO,54,PPL,PARTIDO PÁTRIA LIVRE,111,MÉDICO,1955-04-17 00:00:00,229060740191,53,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,4,NÃO ELEITO,MG,LAMBARI,47554
615,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,LUCIANA RENATA RAMOS,250000008014,15885,29334399805,LUCIANA RAMOS,2,DEFERIDO,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,265,PROFESSOR DE ENSINO FUNDAMENTAL,1978-12-13 00:00:00,255052370159,30,4,FEMININO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,GARÇA,64351


In [111]:
cd08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1227 entries, 0 to 1226
Data columns (total 38 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DT_GERACAO                     1227 non-null   object
 1   HH_GERACAO                     1227 non-null   object
 2   ANO_ELEICAO                    1227 non-null   int64 
 3   NR_TURNO                       1227 non-null   int64 
 4   DS_ELEICAO                     1227 non-null   object
 5   SG_UF                          1227 non-null   object
 6   SG_UE                          1227 non-null   int64 
 7   NM_UE                          1227 non-null   object
 8   CD_CARGO                       1227 non-null   int64 
 9   DS_CARGO                       1227 non-null   object
 10  NM_CANDIDATO                   1227 non-null   object
 11  SQ_CANDIDATO                   1227 non-null   int64 
 12  NR_CANDIDATO                   1227 non-null   int64 
 13  NR_

### 3.2. - Agregar informações:

### 3.2.1 - Agregar informações:
- dados de coligações
- agregação em 'ANO_ELEICAO', 'NR_TURNO', 'SG_UF', 'SG_UE', 'CD_CARGO' e 'NR_PARTIDO'
*https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_legendas/consulta_legendas_2008.zip

#### Informações sobre legendas / coligações
- no ano de 2008, as informações de SG_LEGENDA e COMPOSICAO_LEGENDA não contem nenhum valor na base de dados dos canditados. Apenas as informações de SQ_LEGENDA (número sequencial da legenga) e NM_LEGENDA contem valores.
- apenas tais informações (nome e sequencial) parecem ser pouco úteis para fins de aprendizados de máquina. 
- vamos verificar na base "legendas" quais informações podem agregar informações úteis para a base de canditados
- talvez trazer a informação da composição da legenda e o número de partidos que compõem a legenda ?

In [112]:
def le_coligacao(ano, uf):
    # definição dos nomes de colunas para os anos em o arquivo do TSE não tem os nomes na 1a linha
    c08 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE','DS_UE','CD_CARGO',
       'DS_CARGO','TP_AGREMIACAO','NR_PARTIDO','SG_PARTIDO','NM_PARTIDO','SG_COLIGACAO','NM_COLIGACAO',
       'COMP_COLIGACAO','SQ_COLIGACAO']

    # definição tipos específicos na leitura ininial
    tipos = None
    
    # padroniza nome colunas que serão usadas na junção ou em algum cômputo
    rencols = {'DS_COMPOSICAO_COLIGACAO':'COMP_COLIGACAO'}
    
    if ano > 2012:
        # nome do arquivo zipado baixado do site TSE
        nome_zip = 'tse/candidatos/consulta_coligacao_' + str(ano) + '.zip' 
        # no do arquivo dentro do zip
        arq = 'consulta_coligacao_' + str(ano) + '_' + uf + '.csv'
        cols = None
    else:
        # nome do arquivo zipado baixado do site TSE
        nome_zip = 'tse/candidatos/consulta_legendas_' + str(ano) + '.zip' 
        # no do arquivo dentro do zip
        arq = 'consulta_legendas_' + str(ano) + '_' + uf + '.txt'
        cols = c08
        
    zf = ZipFile(nome_zip)
    df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', names=cols, dtype= tipos)

    # ajustes de nome de coluna que será usada como cômputo
    df.rename(columns=rencols, inplace=True)
    
    # padronização conteúdo coluna usada na agregação para ficar compativel com base de candidatos
    df['DS_ELEICAO'] = df['DS_ELEICAO'].str.title()
    
    # cria nova coluna com a quantidade de partidos na coligação
    df['QT_PART_COL'] = df['COMP_COLIGACAO'].str.count('/')+1
    
    # colunas finais de ineteresse para a agregação
    colsfim = ['ANO_ELEICAO','NR_TURNO', 'DS_ELEICAO', 'SG_UF','SG_UE','CD_CARGO','NR_PARTIDO',
               'TP_AGREMIACAO','SQ_COLIGACAO','NM_COLIGACAO','COMP_COLIGACAO','QT_PART_COL']
    return df[colsfim]

In [113]:
coli08 = le_coligacao(2012,'SP')
coli08.head(2)

,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,CD_CARGO,NR_PARTIDO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL
0,2012,1,Eleição Municipal 2012,SP,61000,11,25,COLIGACAO,250000002861,EUCLIDES EM BOAS MÃOS,PT / PTB / PPS / DEM / PSD,5
1,2012,1,Eleição Municipal 2012,SP,61000,11,55,COLIGACAO,250000002861,EUCLIDES EM BOAS MÃOS,PT / PTB / PPS / DEM / PSD,5


In [114]:
coli08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29293 entries, 0 to 29292
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ANO_ELEICAO     29293 non-null  int64 
 1   NR_TURNO        29293 non-null  int64 
 2   DS_ELEICAO      29293 non-null  object
 3   SG_UF           29293 non-null  object
 4   SG_UE           29293 non-null  int64 
 5   CD_CARGO        29293 non-null  int64 
 6   NR_PARTIDO      29293 non-null  int64 
 7   TP_AGREMIACAO   29293 non-null  object
 8   SQ_COLIGACAO    29293 non-null  int64 
 9   NM_COLIGACAO    29293 non-null  object
 10  COMP_COLIGACAO  29293 non-null  object
 11  QT_PART_COL     29293 non-null  int64 
dtypes: int64(7), object(5)
memory usage: 2.7+ MB


In [115]:
# leva informação das coligações para a base candidatos
cd08 = cd08.merge(coli08, on=['ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF',
                              'SG_UE','CD_CARGO','NR_PARTIDO'], how='left')

In [116]:
cd08.sample(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL
1153,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARIA MARGARIDA ZORDENONI,250000007552,65333,03557003876,MARGARIDA ZORDENONI,2,DEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,298,SERVIDOR PÚBLICO MUNICIPAL,1961-06-20 00:00:00,131559400116,47,4,FEMININO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,ES,RIO BANANAL,57118,PARTIDO ISOLADO,250000000622,#NULO#,#NE#,1
1300,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,ALDA NASCIMENTO BARBARIOLI,250000018946,50881,04908759715,ALDA NASCIMENTO BARBARIOLI,2,DEFERIDO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,298,SERVIDOR PÚBLICO MUNICIPAL,1943-03-06 00:00:00,96133040108,65,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,ES,VIANA,57010,COLIGACAO,250000001276,FRENTE DE ESQUERDA,PCB / PSOL,2
1463,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,TAIS DO NASCIMENTO CHAVES,250000007567,65313,21674567863,TAIS,14,INDEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,126,ANALISTA DE SISTEMAS,1980-11-13 00:00:00,225253380116,28,4,FEMININO,7,SUPERIOR INCOMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,-1,#NULO#,SP,CAMPINAS,62910,PARTIDO ISOLADO,250000000618,#NULO#,#NE#,1


In [117]:
# ajusta os valores NaN que vieram da agregação (partidos que não participam de nenhuma coligação)
# substitui NaN em quantidadde de partidos na coligação para 1 (um único partido, ou seja, sem coligação)
cd08['QT_PART_COL'].fillna(value=1, inplace=True)
# substitui NaN por -1, para o sequencial da coligação.
cd08['SQ_COLIGACAO'].fillna(value=-1, inplace=True)

In [118]:
# ajustar tipo das novas colunas SQ_COLIGACAO e QT_PART_COL de float para int
cd08['QT_PART_COL'] = cd08['QT_PART_COL'].astype(int)
cd08['SQ_COLIGACAO'] = cd08['SQ_COLIGACAO'].astype(int)

#### Conclusões
- confirmadas inconsistências na base candidatos, no que diz respeito a informações de legendas
- na base candidatos, o nome de legenda aperece como nulo para alguns candidatos, mesmo havendo sq_coligacao valido e referencia na base de coligações/legendas
- para Trindade, aparentemente, o unico partido que não participava de nenhuma coligação era o DEM:
    - entretanto, na coluna SQ_COLIGACAO deveria ter o numero -1 (para indicar inexistencia de coligação)
    - mas, ao contrario, aparece o numero 1391 (que não tem registro na base de legendas) 

#### para sanar as inconsistências entendemos que melhor solução será:
- desconsiderar (excluir) todas as colunas referentes a legendas/coligações da base "candidatos":
    - SQ_LEGENDA
    - SG_LEGENDA 
    - COMPOSICAO_LEGENDA
    - NM_LEGENDA
- trazer informações de coligação diretamente da base legendas (a partir do numero do partido e UE) 

#### Buscando informações de coligações/legendas da base legendas
- a partir da base de legendas do TSE, vamos identificar todos os partidos que lançaram candidatos e a respectiva coligação/legenda
- colunas de interesse para adicionar a base de candidatos:
    - TP_AGREMIACAO
    - SQ_COLIGACAO
    - NM_COLIGACAO
    - COMP_COLIGACAO
- alem disso, iremos criar uma nova coluna com a quantidade de partidos presentes na coligação    

#### computando a quantidade de partidos por coligação
- pela coluna COMP_COLIGACAO percebemos que:
- trata-se de um campo com a relação de todos os numeros/partidos que compoem a coligação
- é usado o separador "/" como divisor de partidos
- portanto, vamos calcular a quantidade de partidos pela contagem do caracter "/" somado de 1 

#### Quantidade de partidos na coligação e sq_coligação vieram como float após o merge
- tal fato indica alguma incompatibilidade dos tipos durante a junção
- conforme mostra celulas a seguir, identificamos:
    - 4 linhas NaN para as informações de coligações foram geradas, após o merge
    - tais linhas se referem ao segundo turno da eleição para prefeito em Anápolis
    - não há informações de coligação na base coligações para anapolis/2o turno
    - pode ser uma informação incompleta do próprio TSE
    - como nosso foco não é eleição para prefeito, vamos apenas trocar NaN por -1 nestas 2 colunas
- presença de NaN na coluna qt_partidos_coligação
    - para todas partidos que não participam de coligação retornou NaN
    - neste caso, vamos preencher os Nan com 1 (um partido apenas na "coligação")

### 3.2.2 - Agregar informações:
- quantidade de bens declarados e valor total bens
- agregagação em 'SQ_CANDIDATO'
*https://cdn.tse.jus.br/estatistica/sead/odsele/bem_candidato/bem_candidato_2008.zip

In [119]:
def le_bens(ano, uf):
    c08 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','DS_ELEICAO','SG_UF','SQ_CANDIDATO',
           'CD_TIPO_BEM_CANDIDATO','DS_TIPO_BEM_CANDIDATO','DS_BEM_CANDIDATO',
           'VR_BEM_CANDIDATO','DT_ULTIMA_ATUALIZACAO','HH_ULTIMA_ATUALIZACAO']
    
    nome_zip = 'tse/candidatos/bem_candidato_' + str(ano) + '.zip' 
    tipos = {'ANO_ELEICAO':'string','CD_TIPO_BEM_CANDIDATO':'string'}
    if ano > 2012:
        arq = 'bem_candidato_' + str(ano) + '_' + uf + '.csv'
        cols = None
    else:
        arq = 'bem_candidato_' + str(ano) + '_' + uf + '.txt'
        cols = c08
        
    zf = ZipFile(nome_zip)
    df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', names=cols, dtype=tipos)
    
    # altera coluna valor bem caso ela tenha sido lida como str pelo read_csv
    if type(df['VR_BEM_CANDIDATO'][0]) == str:
        df['VR_BEM_CANDIDATO'] = df['VR_BEM_CANDIDATO'].str.replace(',','.').astype(float)
    
    # cria o df de interesse para agregação, com contagem do total de bens declarados e soma dos valores dos bens
    df1 = df.groupby('SQ_CANDIDATO', as_index=False).agg({'VR_BEM_CANDIDATO':['count','sum']})
    
    # renomeia colunas finais do df de interesse
    df1.columns = ['SQ_CANDIDATO','QT_BENS','VL_TOT_BENS']
    return df1

In [120]:
bens08 = le_bens(2012,'SP')

In [121]:
bens08.sample(3)

,SQ_CANDIDATO,QT_BENS,VL_TOT_BENS
34100,250000068470,4,"161,488.92"
24342,250000049698,1,"50,000.00"
5815,250000014472,4,"236,000.00"


In [122]:
bens08.shape

(50007, 3)

In [123]:
# agregando informações de bens (quantidade e valor total) para a base de candidatos:
cd08 = cd08.merge(bens08, on=['SQ_CANDIDATO'], how='left')

In [124]:
cd08.head(2)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS
0,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARCOS DE ABREU FREIRE,250000082035,13124,08591012828,MARCOS FREIRE,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1966-06-20 00:00:00,98095830183,42,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,2.00,"346,000.00"
1,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARIVALDA SILVA SOARES,250000082043,13773,85656240800,PROF. MARIVALDA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1958-06-22 00:00:00,88188510159,50,4,FEMININO,8,SUPERIOR COMPLETO,7,SEPARADO(A) JUDICIALMENTE,1,BRASILEIRA NATA,5,SUPLENTE,BA,IBIRATAIA,35599,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3.00,"59,746.14"


### 3.2.3 - Agregar informações:
- quantidade de vagas
- terá utilidade na limitação de positivos para o target
- agregação em 'ANO_ELEICAO', 'DS_ELEICAO', 'SG_UF', 'SG_UE' e 'CD_CARGO'
* https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_vagas/consulta_vagas_2008.zip

In [125]:
def le_vagas(ano, uf):
    c08 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','DS_ELEICAO','SG_UF','SG_UE','NM_UE',
           'CD_CARGO','DS_CARGO','QT_VAGAS']
    
    nome_zip = 'tse/candidatos/consulta_vagas_' + str(ano) + '.zip' 
    tipos = {'ANO_ELEICAO':'string'}
    if ano > 2012:
        arq = 'consulta_vagas_' + str(ano) + '_' + uf + '.csv'
        cols = None
    else:
        arq = 'consulta_vagas_' + str(ano) + '_' + uf + '.txt'
        cols = c08
        
    zf = ZipFile(nome_zip)
    df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', names=cols)#, dtype=tipos)
    df['DS_ELEICAO'] = df['DS_ELEICAO'].str.title()
    # colunas finais de interesse para a junção na pase de candidatos
    colsfim = ['ANO_ELEICAO','DS_ELEICAO','SG_UF','SG_UE','CD_CARGO','QT_VAGAS']
    return df[colsfim]

In [126]:
vagas08 = le_vagas(2012,'SP')

In [127]:
vagas08.shape

(1981, 6)

In [128]:
vagas08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1981 entries, 0 to 1980
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ANO_ELEICAO  1981 non-null   int64 
 1   DS_ELEICAO   1981 non-null   object
 2   SG_UF        1981 non-null   object
 3   SG_UE        1981 non-null   int64 
 4   CD_CARGO     1981 non-null   int64 
 5   QT_VAGAS     1981 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 93.0+ KB


In [129]:
vagas08.head(3)

,ANO_ELEICAO,DS_ELEICAO,SG_UF,SG_UE,CD_CARGO,QT_VAGAS
0,2012,Eleição Municipal 2012,SP,65455,13,12
1,2012,Eleição Municipal 2012,SP,65455,12,1
2,2012,Eleição Municipal 2012,SP,65455,11,1


In [130]:
# agregando informações de qtd vagas para a base de candidatos:
cd08 = cd08.merge(vagas08, on=['ANO_ELEICAO','DS_ELEICAO','SG_UF','SG_UE','CD_CARGO'], how='left')

In [131]:
cd08.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS
0,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARCOS DE ABREU FREIRE,250000082035,13124,08591012828,MARCOS FREIRE,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1966-06-20 00:00:00,98095830183,42,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,2.00,"346,000.00",55
1,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARIVALDA SILVA SOARES,250000082043,13773,85656240800,PROF. MARIVALDA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1958-06-22 00:00:00,88188510159,50,4,FEMININO,8,SUPERIOR COMPLETO,7,SEPARADO(A) JUDICIALMENTE,1,BRASILEIRA NATA,5,SUPLENTE,BA,IBIRATAIA,35599,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3.00,"59,746.14",55
2,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,VERA LÚCIA GOULART GONÇALVES,250000082076,13713,64862569820,VERA GOULART,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1953-09-08 00:00:00,149098390116,55,4,FEMININO,6,ENSINO MÉDIO COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,EMILIANÓPOLIS,61387,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3.00,"149,793.95",55


### 3.2.4 - Agregar informações:
- receitas e despesas do candidato da base de prestacoes de contas do TSE
- os dados do TSE trazem as receitas e despesas individualizadas por data, tipo, etc
- nos interessa, num primeiro momento:
    - receitas:
        - totais de receitas por candidato e tipo de receita
    - despesas:
        - valores totais de despesa por candidato
        
- agregação em 'SQ_CANDIDATO'


#### total despadronização das colunas ao longo dos anos
- definição das colunas de nosso interesse, para trazer as informações de receitas e despesas:
    - base recitas:
        - SQ_CANDIDATO: ajustar nomes das colunas nos anos 2008, 2012 e 2016
        - CD_ORIGEM_RECEITA: ajustar nome da coluna no ano 2008 e criar coluna nos anos 2012 e 2016 (só tem coluna com a descrição do tipo da receita)
        - VR_RECEITA: ajustar nome da coluna nos anos 2012 e 2016, tratar de string para float
        
    - base despesas:
        - SQ_CANDIDATO: ajustar nomes das colunas nos anos 2008, 2012 e 2016
        - VR_DESPESA: ajustar nome das colunas nos anos 2012, 2016 e 2020
*https://cdn.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_contas_2008.zip

In [132]:
# função para leitura do arquivo de receitas do candidato
def le_receitas(ano, uf):
    # cria dicionário com os tipos de receitas
    tipos_rec = {'Recursos de pessoas físicas': 10010200,
             'Recursos próprios': 10010100,
             'Recursos de outros candidatos/comitês':10040000,
             'Recursos de outros candidatos':10040000,
             'Recursos de pessoas jurídicas': 10010300,
             'Recursos de partido político': 10020000,
             'Recursos de origens não identificadas': 10030300,
             'Rendimentos de aplicações financeiras': 10030200,
             'Descrição das doações relativas à comercialização': 10050000,
             'Comercialização de bens ou realização de eventos': 10050000,
             'Recursos de Financiamento Coletivo': 10020500,
             'Doações pela Internet': 10010400}
    
    nome_zip = 'tse/contas/prestacao_contas_' + str(ano) + '.zip' 
    tipos = {'SG_UE':'string','CD_NUM_CPF':'string','CD_NUM_CNPJ':'string','DS_NR_TITULO_ELEITOR':'string',
             'CD_CPF_CNPJ_DOADOR':'string','SG_UE_1':'string','CD_CPF_ADM':'string','CPF/CNPJ do doador':'string'}
    zf = ZipFile(nome_zip)
    # padroniza nome colunas que serão usadas na junção
    rencols = {'SEQUENCIAL_CANDIDATO':'SQ_CANDIDATO', 'Sequencial Candidato':'SQ_CANDIDATO',
               'Valor receita':'VR_RECEITA', 'CD_TITULO':'CD_ORIGEM_RECEITA'}
    if ano < 2012:
        arq = 'receitas_candidatos_' + str(ano) + '_brasil.csv'
        iter_csv = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos, iterator=True, chunksize=2000)
        df = pd.concat([chunk[chunk['SG_UE_SUPERIOR'] == uf] for chunk in iter_csv])
    else:
        if ano == 2020:
            arq = 'receitas_candidatos_' + str(ano) + '_' + uf + '.csv' 
            df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos)
        else:
            if ano == 2012:
                arq = 'receitas_candidatos_' + str(ano) + '_' + uf + '.txt' 
            elif ano == 2016:
                arq = 'receitas_candidatos_prestacao_contas_final_' + str(ano) + '_' + uf + '.txt'
            df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos)
            df['CD_ORIGEM_RECEITA']= df['Tipo receita'].map(tipos_rec)
            
    df.rename(columns=rencols, inplace=True)
    # substitui o marcador de centavos (virgula por ponto) na coluna "valor da receita"
    df['VR_RECEITA'] = df['VR_RECEITA'].str.replace(',','.')
    # altera tipo para float
    df['VR_RECEITA'] = df['VR_RECEITA'].astype(float)
    return df 

In [133]:
rec08 = le_receitas(2012,'SP')

In [134]:
rec08.shape

(495374, 23)

In [135]:
rec08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495374 entries, 0 to 495373
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Data e hora                       495374 non-null  object 
 1   SQ_CANDIDATO                      495374 non-null  int64  
 2   UF                                495374 non-null  object 
 3   Numero UE                         495374 non-null  int64  
 4   Municipio                         495374 non-null  object 
 5   Sigla  Partido                    495374 non-null  object 
 6   Numero candidato                  495374 non-null  int64  
 7   Cargo                             495374 non-null  object 
 8   Nome candidato                    495374 non-null  object 
 9   CPF do candidato                  495374 non-null  int64  
 10  Numero Recibo Eleitoral           494633 non-null  object 
 11  Numero do documento               92032 non-null   o

In [136]:
rec08.head(2)

,Data e hora,SQ_CANDIDATO,UF,Numero UE,Municipio,Sigla Partido,Numero candidato,Cargo,Nome candidato,CPF do candidato,Numero Recibo Eleitoral,Numero do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Setor econômico do doador,Data da receita,VR_RECEITA,Tipo receita,Fonte recurso,Especie recurso,Descricao da receita,CD_ORIGEM_RECEITA
0,15/05/201313:44:31,250000028248,SP,71196,SÃO SIMÃO,PV,43043,Vereador,CARLOS EDUARDO BEVILAQUA,25271382842,4304371196SP000005,NaN,00550366000134,WEST AUTO POSTO LTDA.,WEST AUTO POSTO LIMITADA,Comércio varejista de combustíveis para veícul...,06/10/12,250.00,Recursos de pessoas jurídicas,Nao especificado,Estimado,DOAÇÃO DE 140 LITROS DE ALCOOL PARA ABASTECIME...,"10,010,300.00"
1,15/05/201313:44:31,250000094214,SP,68977,POÁ,PDT,12190,Vereador,CLAUDIO DE SIQUEIRA,25840478881,1219068977SP000056,NaN,74277054000135,QUINCOLOR ETIQUETAS ADESIVAS LTDA,QUINCOLOR ETIQUETAS ADESIVAS E ARTES GRAFICAS ...,Impressão de material para outros usos,04/10/12,10.00,Recursos de pessoas jurídicas,Nao especificado,Estimado,ADESIVO MALA DIRETA,"10,010,300.00"


In [137]:
rec08['CD_ORIGEM_RECEITA'].unique()

array([10010300., 10040000., 10020000.,       nan, 10010100.])

In [140]:
rec08[rec08['CD_ORIGEM_RECEITA'].isna()]

,Data e hora,SQ_CANDIDATO,UF,Numero UE,Municipio,Sigla Partido,Numero candidato,Cargo,Nome candidato,CPF do candidato,Numero Recibo Eleitoral,Numero do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Setor econômico do doador,Data da receita,VR_RECEITA,Tipo receita,Fonte recurso,Especie recurso,Descricao da receita,CD_ORIGEM_RECEITA
720,15/05/201313:44:31,250000037035,SP,63614,COTIA,PSL,17456,Vereador,JOSE GENIVAL DOS SANTOS,150273843,1745663614SP000013,306081,04561372000148,WILL ELETROPEÇAS LTDA.,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,10/10/12,200.00,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,"01 CAIXA DE MIDIA DE 4 FALANTES + 4TW, 01 CHIC...",nan
721,15/05/201313:44:31,250000090679,SP,63614,COTIA,PSDB,45,Prefeito,ANTONIO CARLOS DE CAMARGO,3673208861,0004563614SP000080,306089,04561372000148,WILL ELETROPEÇAS LTDA,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,06/10/12,811.50,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,10 PÇS DE CAIXA DE MIDIA 4 FALANTES + 4 TW A R...,nan
722,15/05/201313:44:31,250000090679,SP,63614,COTIA,PSDB,45,Prefeito,ANTONIO CARLOS DE CAMARGO,3673208861,0004563614SP000079,306089,04561372000148,WILL ELETROPEÇAS LTDA,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,06/10/12,811.50,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,10 PÇS DE CAIXA DE MIDIA 4 FALANTES + 4 TW,nan
32792,15/05/201313:44:31,250000043489,SP,70572,SANTO ANDRÉ,PT,13625,Vereador,SILVANA APARECIDA DA SILVA,11959927850,1362570572SP000013,553344000006608,00676262000170,PARTIDO DOS TRABALHADORES PT DIRETORIO NACIONAL,PARTIDO DOS TRABALHADORES,Atividades de organizações políticas,11/09/12,"7,600.00",Recursos de doações pela Internet,Nao especificado,Transferência eletrônica,NaN,nan
32793,15/05/201313:44:31,250000043489,SP,70572,SANTO ANDRÉ,PT,13625,Vereador,SILVANA APARECIDA DA SILVA,11959927850,1362570572SP000024,663344000006608,00676262000170,PARTIDO DOS TRABALHADORES PT DIRETORIO NACIONAL,PARTIDO DOS TRABALHADORES,Atividades de organizações políticas,03/10/12,"9,500.00",Recursos de doações pela Internet,Nao especificado,Transferência eletrônica,NaN,nan


In [144]:
cols = ['Tipo receita','CD_ORIGEM_RECEITA']
rec08[cols].drop_duplicates()

,Tipo receita,CD_ORIGEM_RECEITA
0,Recursos de pessoas jurídicas,"10,010,300.00"
75,Recursos de outros candidatos/comitês,"10,040,000.00"
93,Recursos de partido político,"10,020,000.00"
720,Comercialização de bens e/ou realização de eve...,nan
11097,Recursos próprios,"10,010,100.00"
32792,Recursos de doações pela Internet,nan


In [145]:
rec08['CD_ORIGEM_RECEITA'].fillna(10030300, inplace=True)

In [148]:
rec08['CD_ORIGEM_RECEITA'] = rec08['CD_ORIGEM_RECEITA'].astype(int)

In [149]:
rec08.query('CD_ORIGEM_RECEITA == 10030300')

,Data e hora,SQ_CANDIDATO,UF,Numero UE,Municipio,Sigla Partido,Numero candidato,Cargo,Nome candidato,CPF do candidato,Numero Recibo Eleitoral,Numero do documento,CPF/CNPJ do doador,Nome do doador,Nome do doador (Receita Federal),Setor econômico do doador,Data da receita,VR_RECEITA,Tipo receita,Fonte recurso,Especie recurso,Descricao da receita,CD_ORIGEM_RECEITA
720,15/05/201313:44:31,250000037035,SP,63614,COTIA,PSL,17456,Vereador,JOSE GENIVAL DOS SANTOS,150273843,1745663614SP000013,306081,04561372000148,WILL ELETROPEÇAS LTDA.,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,10/10/12,200.00,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,"01 CAIXA DE MIDIA DE 4 FALANTES + 4TW, 01 CHIC...",10030300
721,15/05/201313:44:31,250000090679,SP,63614,COTIA,PSDB,45,Prefeito,ANTONIO CARLOS DE CAMARGO,3673208861,0004563614SP000080,306089,04561372000148,WILL ELETROPEÇAS LTDA,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,06/10/12,811.50,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,10 PÇS DE CAIXA DE MIDIA 4 FALANTES + 4 TW A R...,10030300
722,15/05/201313:44:31,250000090679,SP,63614,COTIA,PSDB,45,Prefeito,ANTONIO CARLOS DE CAMARGO,3673208861,0004563614SP000079,306089,04561372000148,WILL ELETROPEÇAS LTDA,WILL ELETROPECAS LTDA - ME,Comércio por atacado de peças e acessórios nov...,06/10/12,811.50,Comercialização de bens e/ou realização de eve...,Nao especificado,Cheque,10 PÇS DE CAIXA DE MIDIA 4 FALANTES + 4 TW,10030300
32792,15/05/201313:44:31,250000043489,SP,70572,SANTO ANDRÉ,PT,13625,Vereador,SILVANA APARECIDA DA SILVA,11959927850,1362570572SP000013,553344000006608,00676262000170,PARTIDO DOS TRABALHADORES PT DIRETORIO NACIONAL,PARTIDO DOS TRABALHADORES,Atividades de organizações políticas,11/09/12,"7,600.00",Recursos de doações pela Internet,Nao especificado,Transferência eletrônica,NaN,10030300
32793,15/05/201313:44:31,250000043489,SP,70572,SANTO ANDRÉ,PT,13625,Vereador,SILVANA APARECIDA DA SILVA,11959927850,1362570572SP000024,663344000006608,00676262000170,PARTIDO DOS TRABALHADORES PT DIRETORIO NACIONAL,PARTIDO DOS TRABALHADORES,Atividades de organizações políticas,03/10/12,"9,500.00",Recursos de doações pela Internet,Nao especificado,Transferência eletrônica,NaN,10030300


In [147]:
# função para leitura dos arquivos de despesas dos candidatos
def le_despesas(ano, uf):
    nome_zip = 'tse/contas/prestacao_contas_' + str(ano) + '.zip' 
    tipos = {'ANO_ELEICAO':'string','CD_NUM_CNPJ':'string','CD_TIPO_DOCUMENTO':'string',
             'CD_CPF_CNPJ_FORNECEDOR':'string','SG_UE_1':'string','CD_CPF_ADM':'string',
             'Número do documento':'string','CPF/CNPJ do fornecedor':'string'}
    # padroniza nome colunas que serão usadas na junção
    rencols = {'SEQUENCIAL_CANDIDATO':'SQ_CANDIDATO', 'Sequencial Candidato':'SQ_CANDIDATO',
               'Valor despesa':'VR_DESPESA', 'VR_DESPESA_CONTRATADA':'VR_DESPESA'}
    zf = ZipFile(nome_zip)

    if ano < 2012:
        arq = 'despesas_candidatos_' + str(ano) + '_brasil.csv'
        iter_csv = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos, iterator=True, chunksize=2000)
        df = pd.concat([chunk[chunk['SG_UE_SUPERIOR'] == uf] for chunk in iter_csv])
        
    else:
        if ano == 2012:
            arq = 'despesas_candidatos_' + str(ano) + '_' + uf + '.txt' 
        elif ano == 2016:
            arq = 'despesas_candidatos_prestacao_contas_final_' + str(ano) + '_' + uf + '.txt'
        elif ano == 2020:
            arq = 'despesas_contratadas_candidatos_' + str(ano) + '_' + uf + '.csv'
        df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos)
    df.rename(columns=rencols, inplace=True)
    # ajusta coluna de valores de str para float
    df['VR_DESPESA'] = df['VR_DESPESA'].str.replace(',','.')
    df['VR_DESPESA'] = df['VR_DESPESA'].astype(float)
    return df 

In [150]:
desp08 = le_despesas(2012,'SP')

In [151]:
desp08.shape

(619739, 20)

In [152]:
desp08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619739 entries, 0 to 619738
Data columns (total 20 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Data e hora                           619739 non-null  object 
 1   SQ_CANDIDATO                          619739 non-null  int64  
 2   UF                                    619739 non-null  object 
 3   Número UE                             619739 non-null  int64  
 4   Município                             619739 non-null  object 
 5   Sigla  Partido                        619739 non-null  object 
 6   Número candidato                      619739 non-null  int64  
 7   Cargo                                 619739 non-null  object 
 8   Nome candidato                        619739 non-null  object 
 9   CPF do candidato                      619739 non-null  int64  
 10  Tipo do documento                     615376 non-null  object 
 11  

In [153]:
desp08.head(2)

,Data e hora,SQ_CANDIDATO,UF,Número UE,Município,Sigla Partido,Número candidato,Cargo,Nome candidato,CPF do candidato,Tipo do documento,Número do documento,CPF/CNPJ do fornecedor,Nome do fornecedor,Nome do fornecedor (Receita Federal),Setor econômico do fornecedor,Data da despesa,VR_DESPESA,Tipo despesa,Descriçao da despesa
0,15/05/201313:59:39,250000083753,SP,61751,ARTUR NOGUEIRA,PSDB,45200,Vereador,AIRTON BARBOSA,92370160853,Nota Fiscal,1416,00487300000147,GRAFICA LAGOA BONITA LTDA-ME,GRAFICA LAGOA BONITA LTDA ME,Impressão de material para uso publicitário,28/08/12,130.00,Publicidade por materiais impressos,5.000 FOLHETOS FORMATO 10X15CM EM SULFITE-IMPR...
1,15/05/201313:59:39,250000041480,SP,62570,BRODOWSKI,PT,13,Prefeito,ELVES SCIARRETA CARREIRA,4362630848,Nota Fiscal,1121,00281049000160,LBR AUTO POSTO LTDA,L B R AUTO POSTO LTDA,Comércio varejista de combustíveis para veícul...,10/10/12,"8,264.39",Combustíveis e lubrificantes,COMBUSTIVEIS E LUBRIFICANTES (NF EMITIDA CFOP ...


In [155]:
desp08['Tipo despesa'].unique()

array(['Publicidade por materiais impressos',
       'Combustíveis e lubrificantes',
       'Publicidade por jornais e revistas', 'Diversas a especificar',
       'Publicidade por placas, estandartes e faixas',
       'Materiais de expediente', 'Serviços prestados por terceiros',
       'Cessão ou locação de veículos', 'Publicidade por carros de som',
       'Eventos de promoção da candidatura',
       'Produção de jingles, vinhetas e slogans',
       'Produção de programas de rádio, televisão ou vídeo',
       'Despesas com pessoal', 'Correspondências e despesas postais',
       'Comícios', 'Criação e inclusão de páginas na internet',
       'Locação/cessão de bens imóveis', 'Locação/cessão de bens móveis',
       'Pesquisas ou testes eleitorais', 'Publicidade por telemarketing',
       'Alimentação', 'Água', 'Bens permanentes', 'Energia elétrica',
       'Telefone',
       'Pré-instalação física de comitê de campanha de candidato',
       'Despesas com transporte ou deslocamento',
  

In [156]:
def cria_receita(nome_col,lst_cod_rec):
    # colunas de ineteresse no dataset de receitas
    cols = ['SQ_CANDIDATO', 'VR_RECEITA']
    
    # agrupa (soma) as receitas por candidato e codigo de receita
    temp = rec08.groupby(['SQ_CANDIDATO','CD_ORIGEM_RECEITA'], as_index=False).agg({'VR_RECEITA':'sum'})
    
    # filtra apenas os códigos de receita escolhidos(no parametro da função) e agrupa (soma) por cnadidato
    df = temp.query('CD_ORIGEM_RECEITA in @lst_cod_rec').groupby('SQ_CANDIDATO', as_index=False).sum('VR_RECEITA')[cols]
    
    # renomeia a coluna valor da receita para o nome definido na função (parametro)
    df.rename(columns = {'VR_RECEITA':nome_col}, inplace=True)
    return df

In [157]:
r_prop = cria_receita('REC_PROP',(10010100,))
r_pf = cria_receita('REC_PF',(10010200,))
r_pj = cria_receita('REC_PJ',(10010300,))
r_partcand = cria_receita('REC_PARTCAND',(10020000,10040000))
r_web = cria_receita('REC_WEB',(10010400,10020500))
r_out = cria_receita('REC_OUTRAS',(10030200,10030300,10050000))

In [161]:
r_prop.sample(3)

,SQ_CANDIDATO,REC_PROP
19298,250000051484,"3,614.00"
36689,250000095541,450.00
18653,250000049845,"3,936.40"


In [162]:
# despesas totais
cols = ['SQ_CANDIDATO', 'VR_DESPESA']
d08_tot = desp08.groupby('SQ_CANDIDATO', as_index=False).sum('VR_DESPESA')[cols]

In [163]:
# agregando as novas colunas a base de dados de candidatos
cd08 = cd08.merge(r_prop, on=['SQ_CANDIDATO'], how='left')
cd08 = cd08.merge(r_pf, on=['SQ_CANDIDATO'], how='left')
cd08 = cd08.merge(r_pj, on=['SQ_CANDIDATO'], how='left')
cd08 = cd08.merge(r_partcand, on=['SQ_CANDIDATO'], how='left')
#cd08 = cd08.merge(r_web, on=['SQ_CANDIDATO'], how='left')
cd08 = cd08.merge(r_out, on=['SQ_CANDIDATO'], how='left')
cd08 = cd08.merge(d08_tot, on=['SQ_CANDIDATO'], how='left')

In [164]:
cd08.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA
1106,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,CARLOS ROGÉRIO FREIHAT HENRIQUE,250000007500,65533,08823572800,ROGÉRIO FREIHAT,2,DEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,298,SERVIDOR PÚBLICO MUNICIPAL,1966-12-03 00:00:00,85155280141,42,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,PARTIDO ISOLADO,250000000618,#NULO#,#NE#,1,3.00,"37,044.16",55,"2,720.00",nan,nan,"5,109.56",nan,"7,999.56"
742,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,RENALDO ROQUE DOS SANTOS,250000002395,44150,71646256468,REGIS DO FORRÓ,2,DEFERIDO,44,PRP,PARTIDO REPUBLICANO PROGRESSISTA,163,CANTOR E COMPOSITOR,1969-02-10 00:00:00,268253180183,39,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,AL,GIRAU DO PONCIANO,27570,COLIGACAO,250000000277,UNIDOS POR SÃO PAULO,PHS / PRP,2,nan,nan,55,nan,nan,nan,"1,822.64",nan,"1,822.64"
883,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,DEBORAH ESTER VERISSIMO CAMARGO,250000002232,43699,05493039877,DEBORAH VERISSIMO,2,DEFERIDO,43,PV,PARTIDO VERDE,131,ADVOGADO,1963-12-18 00:00:00,91515020159,45,4,FEMININO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,PARTIDO ISOLADO,250000006581,#NULO#,#NE#,1,nan,nan,55,nan,nan,nan,"4,521.20",nan,"4,521.20"
897,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,SANDRA APARECIDA PANDOLFI COSTA,250000002291,43040,26297858829,SANDRA PANDOLFI,2,DEFERIDO,43,PV,PARTIDO VERDE,999,OUTROS,1974-07-13 00:00:00,268892900132,34,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,ITAPECERICA DA SERRA,65455,PARTIDO ISOLADO,250000006581,#NULO#,#NE#,1,2.00,"311,000.00",55,"1,050.00",nan,nan,"2,054.60",nan,"3,061.60"
25,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,VERA LUCIA MOREIRA DA SILVA,250000082075,13171,04529096866,VERA MOREIRA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1963-12-19 00:00:00,151341530124,45,4,FEMININO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,5.00,"55,864.68",55,nan,nan,nan,"20,290.79",nan,"20,290.79"


#### Impossibilidade de uso do groupby com unstack para gerar uma coluna para cada tipo de receita
- como nem todo candidato tem todos os tipos de receitas, as colunas geradas não ficam com a ordem adequada
- faremos a extração dos valores de receita de forma individualizada, para cada codigo que queremos adicionar a base de candidatos:
    - 10010100 recursos proprios
    - 10010200 + 10010300 recursos oriundos de doações de PF + PJ
    - 10020000 + 10040000 recursos de partidos politicos + recursos de outros candidatos / legendas
    - 10010400 + 10020500 doacoes internet + financiamento coletivo
    - outras: 10030200 + 10030300

#### Análise das despesas
- estão separadas por tipo assim como as receitas
- num primeiro momento, não nos pareceu util trazer despesas separadas por tipos para fins de agregação na base de candidatos
- vamos trazer apenas o total de despesas

#### Dados que serão trazidos da base de prestação de contas
- despesas:
    - despesa total 

- receitas:
    - recursos proprios (10010100,)
    - recursos doações PF + PJ (10010200,10010300)
    - recursos partidos politicos + outros candidatos / legendas (10020000,10040000)
    - recursos doacoes internet + financiamento coletivo (10010400,10020500)
    - outras: (10030200,10030300)

### 3.2.5 - Agregar informações:
- base de resultados da legislatura anterior
    - total de votos do partido por cargo na eleição anterior
    - total de cadeiras por cargo conquistadas pelo partido na eleição anterior

- pasta dos arquivos: tse/resultados/

- nome arquivo zipado:
    - 2004 a 2020: votacao_candidato_munzona_AAAA.zip

- nome arquivo final:
    - até 2012: votacao_candidato_munzona_AAAA_UF.txt
    - a partir de 2014: votacao_candidato_munzona_AAAA_UF.csv

- colunas usadas para agregação:
    - 'NR_TURNO'
    - 'SG_UF'
    - 'SG_UE'
    - 'CD_CARGO'
    - 'NR_PARTIDO'
    
- outras colunas usadas e que precisam padronização:
    - DS_SIT_T_CAND -> usada para definir a nova coluna ELEITO
        - precisa padronização a partir do ano de 2014, que passa a se chamar DS_SIT_TOT_TURNO
        - manteremos o nome mais recente
    - agregação para somar votos por partido / cargo até 2012:
        - 'ANO_ELEICAO'
        - 'NR_TURNO'
        - 'DS_ELEICAO'
        - 'SG_UF'
        - 'SG_UE'
        - 'CD_CARGO'
        - 'NR_PARTIDO'
        - 'SQ_CANDIDATO'
        - 'SQ_LEGENDA'
        - 'ELEITO'
    - agregação para somar votos por partido / cargo a partir 2014:
        - alem de todos as colunas anteriores, acrestar a coluna CD_TIPO_ELEICAO
        - nos anos anteriores oo tipo da eleição (se ordinária ou extraordina) está na própria descrição
        
 *https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_candidato_munzona/votacao_candidato_munzona_2004.zip


In [ ]:
# cria coluna 'ELEITO' valores 1 para sim e 0 para não
#totalizando votos por candidatos (agrega zonas)
#totalizando votos por partidos e total de eleitos
# agrega dados de quantidades de votos do partido e eleitos pelo partido na legislatura atual
# troca nome das colunas agregadas

In [165]:
# função para leitura dos resultados da eleição anterior
def le_eleicaoanterior(ano, uf):
    
    # definição dos nomes de colunas para os anos em o arquivo do TSE não os nomes na 1a linha
    c08 = ['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE','CD_MUN',
       'NM_UE','NR_ZONA','CD_CARGO','NR_CAND','SQ_CANDIDATO','NM_CAND','NM_URNA_CAND','DS_CARGO',
       'CD_SIT_CAND_SUP','DS_SIT_CAND_SUP','CD_SIT_REG_CAND','DS_SIT_REF_CAND','CD_SIT_T_CAND','DS_SIT_T_CAND','NR_PARTIDO','SG_PARTIDO','NM_PARTIDO',
       'SQ_LEGENDA','NM_LEGENDA','COMP_LEG','TOT_VOTOS']
    
    # nome do arquivo zipado baixado do site TSE
    nome_zip = 'tse/resultados/votacao_candidato_munzona_' + str(ano) + '.zip' 
    zf = ZipFile(nome_zip)
    
    # nome do arquivo final dentro do zip
    arq = 'votacao_candidato_munzona_' + str(ano) + '_' + uf + '.csv'
    
    # definição tipos específicos na leitura ininial
    tipos = None #{'ANO_ELEICAO':'string'}
    
    # padroniza nome colunas que serão usadas na junção ou em algum cômputo
    rencols = {'DS_SIT_T_CAND':'DS_SIT_TOT_TURNO', 'SQ_LEGENDA':'SQ_COLIGACAO',
               'QT_VOTOS_NOMINAIS':'TOT_VOTOS'}

    # definição das colunas agregadoras, conforme ano do arquivo
    ag08 = ['ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE','NM_UE','CD_CARGO',
            'NR_PARTIDO','SQ_CANDIDATO','SQ_COLIGACAO','ELEITO']

    ag14 = ['ANO_ELEICAO','CD_TIPO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE',
            'NM_UE','CD_CARGO','NR_PARTIDO','SQ_CANDIDATO','SQ_COLIGACAO','ELEITO']

    if ano < 2014:
        # nome do arquivo final dentro do zip
        arq = 'votacao_candidato_munzona_' + str(ano) + '_' + uf + '.txt'
        cols = c08
        ag = ag08
    else:
        # nome do arquivo final dentro do zip
        arq = 'votacao_candidato_munzona_' + str(ano) + '_' + uf + '.csv'
        cols = None
        ag = ag14
    df = pd.read_csv(zf.open(arq), sep=';', encoding='latin_1', dtype=tipos, names=cols)
    
    # ajustes conteúdo de coluna que será usada como agregador
    df['DS_ELEICAO'] = df['DS_ELEICAO'].str.title()

    # ajustes de nome de coluna que será usada como cômputo
    df.rename(columns=rencols, inplace=True)
    
    # cria coluna 'ELEITO' valores 1 para sim e 0 para não
    df['ELEITO'] = np.where((df['DS_SIT_TOT_TURNO'] == "ELEITO") |
                            (df['DS_SIT_TOT_TURNO'] == "MÉDIA") |
                            (df['DS_SIT_TOT_TURNO'] == "ELEITO POR QP") |
                            (df['DS_SIT_TOT_TURNO'] == "ELEITO POR MÉDIA"), 1, 0)
    
    # tabela de agregação temporária
    df1 = df.groupby(ag, as_index=False).agg({'TOT_VOTOS':'sum'})
    
    # colunas para agregação final
    agfim = ['ANO_ELEICAO','NR_TURNO','DS_ELEICAO','SG_UF','SG_UE',
            'CD_CARGO','NR_PARTIDO']    
    
    #colunas finais para o DF
    colsfim = ['NR_TURNO','SG_UF','SG_UE','CD_CARGO',
               'NR_PARTIDO','TOT_VOTOS','ELEITO']

    df2 = df1.groupby(agfim, as_index=False).agg({'TOT_VOTOS':'sum','ELEITO':'sum'})[colsfim]
    df2.rename(columns={'TOT_VOTOS':'VOTOS_PART_ELEC_ANT', 'ELEITO':'QT_ELEITOS_ELEC_ANT'}, inplace=True)

    return df2

In [166]:
leg04 = le_eleicaoanterior(2008,'SP')

In [167]:
leg04.sample(5)

,NR_TURNO,SG_UF,SG_UE,CD_CARGO,NR_PARTIDO,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT
8241,1,SP,70513,13,45,2183,1
6255,1,SP,67814,13,40,155,1
5204,1,SP,66397,13,25,3557,0
4396,1,SP,65471,13,27,1266,0
7281,1,SP,69175,13,33,581,1


In [168]:
leg04.shape

(9930, 7)

In [169]:
leg04.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9930 entries, 0 to 9929
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NR_TURNO             9930 non-null   int64 
 1   SG_UF                9930 non-null   object
 2   SG_UE                9930 non-null   int64 
 3   CD_CARGO             9930 non-null   int64 
 4   NR_PARTIDO           9930 non-null   int64 
 5   VOTOS_PART_ELEC_ANT  9930 non-null   int64 
 6   QT_ELEITOS_ELEC_ANT  9930 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 620.6+ KB


In [170]:
# agrega dados de quantidades de votos do partido e eleitos pelo partido na legislatura atual
cd08 = cd08.merge(leg04, on=['NR_TURNO','SG_UF','SG_UE','CD_CARGO','NR_PARTIDO'], how='left')

In [171]:
cd08.sample(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT
456,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,ESIDIO GODINHO,250000039072,10555,85552178815,BELÊ BELÊ,2,DEFERIDO,10,PRB,PARTIDO REPUBLICANO BRASILEIRO,703,ELETRICISTA E ASSEMELHADOS,1957-12-11 00:00:00,98061350167,51,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SALMOURÃO,70033,COLIGACAO,250000002541,'PRB-PTB',PRB / PTB,2,5.00,"522,000.00",55,"3,506.80",nan,240.00,"2,828.01",nan,"6,454.81","33,235.00",1.00
1108,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARIA DA CONCEIÇÃO COSTA FERREIRA NUNES DE OLI...,250000007548,65245,14700204320,CONCEIÇÃO,2,DEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,297,SERVIDOR PÚBLICO ESTADUAL,1957-12-28 00:00:00,94219520141,51,4,FEMININO,8,SUPERIOR COMPLETO,5,VIÚVO(A),1,BRASILEIRA NATA,5,SUPLENTE,MA,MATINHA,8290,PARTIDO ISOLADO,250000000622,#NULO#,#NE#,1,4.00,"66,318.00",55,nan,nan,100.00,nan,nan,45.00,"159,153.00",2.00
884,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,ERALDO SILVA FILHO,250000002237,43643,16989470895,ERALDO ESSE É DA COMUNIDADE,2,DEFERIDO,43,PV,PARTIDO VERDE,999,OUTROS,1969-01-19 00:00:00,171372310108,39,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,AL,PENEDO,28339,PARTIDO ISOLADO,250000000274,#NULO#,#NE#,1,nan,nan,55,nan,nan,"8,445.00","2,987.80",nan,"10,437.80","329,117.00",3.00


### 3.2.7 - Tratar NaN nas colunas trazidas de outras fontes de dados
- colunas: QT_BENS, VL_TOT_BENS, VR_DESPESA, REC_PROP, REC_PF, REC_PJ, REC_PARTCAND, REC_WEB, REC_OUTRAS
    - considerar NaN como ZERO, sem prejuízo, em principio, para a análise pois trata de valores informados pelos candidatos e os campos NaN indicam que o candidato não informou valores.

- colunas: VOTOS_PART_ELEC_ANT	e QT_ELEITOS_ELEC_ANT
    - em tese são, de fato, partidos que não tiveram votos e eleitos na eleição anterior. considerar ZERO
- em resumo: todas as colunas que ainda constem NaN podem ser preenchidas com ZERO sem prejuizo para a análise
    

In [172]:
cd08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1480 entries, 0 to 1479
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   DT_GERACAO                     1480 non-null   object 
 1   HH_GERACAO                     1480 non-null   object 
 2   ANO_ELEICAO                    1480 non-null   int64  
 3   NR_TURNO                       1480 non-null   int64  
 4   DS_ELEICAO                     1480 non-null   object 
 5   SG_UF                          1480 non-null   object 
 6   SG_UE                          1480 non-null   int64  
 7   NM_UE                          1480 non-null   object 
 8   CD_CARGO                       1480 non-null   int64  
 9   DS_CARGO                       1480 non-null   object 
 10  NM_CANDIDATO                   1480 non-null   object 
 11  SQ_CANDIDATO                   1480 non-null   int64  
 12  NR_CANDIDATO                   1480 non-null   i

In [173]:
cd08.isnull().any()

DT_GERACAO             False
HH_GERACAO             False
ANO_ELEICAO            False
NR_TURNO               False
DS_ELEICAO             False
                       ...  
REC_PARTCAND            True
REC_OUTRAS              True
VR_DESPESA              True
VOTOS_PART_ELEC_ANT     True
QT_ELEITOS_ELEC_ANT     True
Length: 54, dtype: bool

In [174]:
# substituindo NaN por ZERO no dataframe
cd08.fillna(0, inplace=True)

In [175]:
# ajustando tipo das colunas VOTOS_PART_CARGO, QT_ELEITOS_CARGO e qd_bens para integer 
cd08['VOTOS_PART_ELEC_ANT'] = cd08['VOTOS_PART_ELEC_ANT'].astype(int)
cd08['QT_ELEITOS_ELEC_ANT'] = cd08['QT_ELEITOS_ELEC_ANT'].astype(int)
cd08['QT_BENS'] = cd08['QT_BENS'].astype(int)

In [176]:
cd08.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,NR_TURNO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT
0,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARCOS DE ABREU FREIRE,250000082035,13124,08591012828,MARCOS FREIRE,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1966-06-20 00:00:00,98095830183,42,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,2,"346,000.00",55,"6,900.00",0.00,0.00,"47,713.76",0.00,"41,540.45",736408,11
1,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MARIVALDA SILVA SOARES,250000082043,13773,85656240800,PROF. MARIVALDA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1958-06-22 00:00:00,88188510159,50,4,FEMININO,8,SUPERIOR COMPLETO,7,SEPARADO(A) JUDICIALMENTE,1,BRASILEIRA NATA,5,SUPLENTE,BA,IBIRATAIA,35599,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3,"59,746.14",55,"1,650.00",0.00,0.00,"48,825.06",0.00,"36,947.30",736408,11
2,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,VERA LÚCIA GOULART GONÇALVES,250000082076,13713,64862569820,VERA GOULART,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1953-09-08 00:00:00,149098390116,55,4,FEMININO,6,ENSINO MÉDIO COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,EMILIANÓPOLIS,61387,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3,"149,793.95",55,"9,250.00",0.00,0.00,"52,950.44",0.00,"39,561.80",736408,11
3,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,MANOEL DEL RIO BLAS FILHO,250000082030,13313,12905321849,MANOEL DEL RIO,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,131,ADVOGADO,1947-02-03 00:00:00,94750840108,61,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,NIPOÃ,67555,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,7,"527,346.67",55,"27,035.00",0.00,"40,000.00","213,830.54",0.00,"143,085.36",736408,11
4,15/07/2016,19:25:35,2012,1,Eleição Municipal 2012,SP,71072,SÃO PAULO,13,VEREADOR,VERA EUNICE RODRIGUES DA SILVA,250000082074,13678,10461340860,VERINHA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1968-10-24 00:00:00,187721990183,40,4,FEMININO,5,ENSINO MÉDIO INCOMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,0,0.00,55,"1,500.00",0.00,"21,550.00","173,330.54",0.00,"116,506.72",736408,11


### 3.3.  Outros Ajustes no Dataset

### 3.3.1.  Exclusão colunas desnecessárias

In [177]:
for col in cd08.columns:
    if cd08[col].nunique() == 1:
        print(col, cd08[col].unique())

DT_GERACAO ['15/07/2016']
HH_GERACAO ['19:25:35']
ANO_ELEICAO [2012]
NR_TURNO [1]
DS_ELEICAO ['Eleição Municipal 2012']
SG_UF ['SP']
SG_UE [71072]
NM_UE ['SÃO PAULO']
CD_CARGO [13]
DS_CARGO ['VEREADOR']
QT_VAGAS [55]
REC_PF [0.]
REC_OUTRAS [0.]


In [178]:
cd08.nunique().sort_values().head(14)

DT_GERACAO       1
HH_GERACAO       1
ANO_ELEICAO      1
NR_TURNO         1
DS_ELEICAO       1
SG_UF            1
SG_UE            1
NM_UE            1
CD_CARGO         1
DS_CARGO         1
REC_OUTRAS       1
REC_PF           1
QT_VAGAS         1
TP_AGREMIACAO    2
Length: 14, dtype: int64

In [200]:
# exclusão colunas desnecessárias
rv08 = cd08.drop(columns=['DT_GERACAO','HH_GERACAO','ANO_ELEICAO','NR_TURNO',
                          'DS_ELEICAO','SG_UF','CD_CARGO','DS_CARGO'])

In [201]:
rv08.head(2)

,SG_UE,NM_UE,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT
0,71072,SÃO PAULO,MARCOS DE ABREU FREIRE,250000082035,13124,08591012828,MARCOS FREIRE,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1966-06-20 00:00:00,98095830183,42,2,MASCULINO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,2,"346,000.00",55,"6,900.00",0.00,0.00,"47,713.76",0.00,"41,540.45",736408,11
1,71072,SÃO PAULO,MARIVALDA SILVA SOARES,250000082043,13773,85656240800,PROF. MARIVALDA,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1958-06-22 00:00:00,88188510159,50,4,FEMININO,8,SUPERIOR COMPLETO,7,SEPARADO(A) JUDICIALMENTE,1,BRASILEIRA NATA,5,SUPLENTE,BA,IBIRATAIA,35599,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,3,"59,746.14",55,"1,650.00",0.00,0.00,"48,825.06",0.00,"36,947.30",736408,11


### 3.3.2. Exclusão de registros de candidaturas não confirmadas
- para nossa análise não interessa registros de candidaturas inválidos.
- 2 colunas tratam da situação da candidatura:
    - DS_SITUACAO_CANDIDATURA e
    - DS_SIT_TOT_TURNO
- inicialmente vamos filtrar apenas candidaturas DEFERIDA, descartando portanto as INDEFERIDAS conforme coluna DS_SITUACAO_CANDIDATURA

- excluir da base canditados outros eventuais registros inválidos, conforme coluna confirmada DS_SIT_TOT_TURNO
    

In [202]:
# tipos de "situação de candidatura"
rv08['DS_SITUACAO_CANDIDATURA'].unique()

array(['DEFERIDO', 'INDEFERIDO COM RECURSO', 'RENÚNCIA', 'INDEFERIDO'],
      dtype=object)

In [203]:
# quantidade de registros e colunas do dataset 
rv08.shape

(1480, 46)

In [204]:
# nos interessa apenas os registros deferidos ("DEFERIDO" e "DEFERIDO COM RECURSO")
rv08 = rv08.query('DS_SITUACAO_CANDIDATURA in ("DEFERIDO COM RECURSO","DEFERIDO")')

In [205]:
# quantidade de registros e colunas do dataset após exclusão das candidaturas indeferidas
rv08.shape

(1375, 46)

In [206]:
# tipos de "situação de totalização ao final do turno"
rv08['DS_SIT_TOT_TURNO'].unique()

array(['SUPLENTE', 'ELEITO POR MÉDIA', 'ELEITO POR QP', 'NÃO ELEITO'],
      dtype=object)

In [207]:
# nos interessa apenas as situações de candidaturas válidas
# devemos excluir tambem as situações de 'INDEFERIDO COM RECURSO'
rv08.query('DS_SIT_TOT_TURNO == "INDEFERIDO COM RECURSO"')

,SG_UE,NM_UE,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT


In [208]:
rv08 = rv08.query('DS_SIT_TOT_TURNO in ("NÃO ELEITO","SUPLENTE","ELEITO","MÉDIA","ELEITO POR MÉDIA","ELEITO POR QP")')

In [209]:
# quantidade de registros e colunas do dataset após exclusão das candidaturas indeferidas
rv08.shape

(1375, 46)

In [210]:
rv08.sample(3)

,SG_UE,NM_UE,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT
32,71072,SÃO PAULO,ROBERTA CRISTINA NININ,250000082061,13413,31158096895,ROBERTA NININ,2,DEFERIDO,13,PT,PARTIDO DOS TRABALHADORES,999,OUTROS,1983-05-05 00:00:00,304580180108,25,4,FEMININO,8,SUPERIOR COMPLETO,1,SOLTEIRO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,MARÍLIA,66818,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,0,0.00,55,0.00,0.00,0.00,"27,126.54",0.00,"26,927.04",736408,11
100,71072,SÃO PAULO,RICARDO NICASTRO,250000082059,40087,07031766842,RICARDO NICASTRO,2,DEFERIDO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,999,OUTROS,1967-11-05 00:00:00,130058870141,41,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,PRESIDENTE PRUDENTE,69299,COLIGACAO,250000005140,PARA RENOVAR SÃO PAULO,PP / PT / PSB,3,0,0.00,55,0.00,0.00,0.00,0.00,0.00,0.00,77028,2
372,71072,SÃO PAULO,LUIZ CARLOS BENTO FERREIRA,250000039103,14550,05208395878,LUIZINHO,2,DEFERIDO,14,PTB,PARTIDO TRABALHISTA BRASILEIRO,257,EMPRESÁRIO,1963-01-08 00:00:00,113953210167,45,2,MASCULINO,5,ENSINO MÉDIO INCOMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,SP,SÃO PAULO,71072,COLIGACAO,250000002541,'PRB-PTB',PRB / PTB,2,12,"428,905.47",55,0.00,0.00,0.00,0.00,0.00,0.00,305236,3


### 3.3.3. Criação da coluna resultado (target)
- a base para criação da coluna target será a coluna DS_SIT_TOT_TURNO:
- situação ELEITO e MÉDIA são os SIM (eleitos)
- demais são os NÃO (NÃO ELEITO e SUPLENTE)

In [211]:
# cria coluna alvo: 'RESULTADO' com valores 1 para ELEITO (sim) e 0 para NÃO ELEITO (não)
rv08['RESULTADO'] = np.where((rv08['DS_SIT_TOT_TURNO'] == "ELEITO") |
                             (rv08['DS_SIT_TOT_TURNO'] == "MÉDIA") |
                             (rv08['DS_SIT_TOT_TURNO'] == "ELEITO POR QP") | 
                             (rv08['DS_SIT_TOT_TURNO'] == "ELEITO POR MÉDIA"), 1, 0)

In [212]:
rv08.sample(3)

,SG_UE,NM_UE,NM_CANDIDATO,SQ_CANDIDATO,NR_CANDIDATO,NR_CPF_CANDIDATO,NM_URNA_CANDIDATO,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,CD_OCUPACAO,DS_OCUPACAO,DT_NASCIMENTO,NR_TITULO_ELEITORAL_CANDIDATO,IDADE_DATA_POSSE,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_NACIONALIDADE,DS_NACIONALIDADE,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,SG_UF_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,SQ_COLIGACAO,NM_COLIGACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,QT_VAGAS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT,RESULTADO
1324,71072,SÃO PAULO,WILSON GUANAIS,250000017671,12195,25987949823,WILSON GUANAIS,2,DEFERIDO,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,185,ESCRITOR E CRÍTICO,1972-05-19 00:00:00,244749480159,36,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,4,NÃO ELEITO,SP,TUPÃ,72010,PARTIDO ISOLADO,250000001207,#NULO#,#NE#,1,0,0.00,55,0.00,0.00,0.00,0.00,0.00,0.00,143901,1,0
764,71072,SÃO PAULO,GENIVALDO FERNANDES MOREIRA,250000002345,44155,95124799668,MINEIRO,2,DEFERIDO,44,PRP,PARTIDO REPUBLICANO PROGRESSISTA,169,COMERCIANTE,1973-10-08 00:00:00,104004260264,35,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRASILEIRA NATA,5,SUPLENTE,MG,ITACARAMBI,46418,COLIGACAO,250000000277,UNIDOS POR SÃO PAULO,PHS / PRP,2,0,0.00,55,0.00,0.00,0.00,"11,488.64",0.00,"11,488.64",34743,0,0
877,71072,SÃO PAULO,CARLOS ALBERTO FARIA DE ARAUJO,250000002225,43345,29818309880,CARLOS ARAUJO,2,DEFERIDO,43,PV,PARTIDO VERDE,172,PUBLICITÁRIO,1941-10-06 00:00:00,227542960175,67,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRASILEIRA NATA,5,SUPLENTE,RJ,NITERÓI,58653,PARTIDO ISOLADO,250000006581,#NULO#,#NE#,1,0,0.00,55,358.80,0.00,0.00,0.00,0.00,358.80,329117,3,0


### 3.3.4. Exclusão colunas redundantes e sem utilidade preditiva
- excluir colunas sem informações relevantes para fins preditivos
- pares de colunas redundantes: coluna de código / coluna de descrição


In [192]:
rv08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314 entries, 0 to 1374
Data columns (total 47 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SG_UE                          1314 non-null   int64  
 1   NM_UE                          1314 non-null   object 
 2   NM_CANDIDATO                   1314 non-null   object 
 3   SQ_CANDIDATO                   1314 non-null   int64  
 4   NR_CANDIDATO                   1314 non-null   int64  
 5   NR_CPF_CANDIDATO               1314 non-null   string 
 6   NM_URNA_CANDIDATO              1314 non-null   object 
 7   CD_SITUACAO_CANDIDATURA        1314 non-null   int64  
 8   DS_SITUACAO_CANDIDATURA        1314 non-null   object 
 9   NR_PARTIDO                     1314 non-null   int64  
 10  SG_PARTIDO                     1314 non-null   object 
 11  NM_PARTIDO                     1314 non-null   object 
 12  CD_OCUPACAO                    1314 non-null   i

In [197]:
rv08.columns

Index(['SG_UE', 'NM_UE', 'NM_CANDIDATO', 'SQ_CANDIDATO', 'NR_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NM_URNA_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO',
       'CD_OCUPACAO', 'DS_OCUPACAO', 'DT_NASCIMENTO',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'IDADE_DATA_POSSE', 'CD_GENERO',
       'DS_GENERO', 'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO',
       'CD_ESTADO_CIVIL', 'DS_ESTADO_CIVIL', 'CD_NACIONALIDADE',
       'DS_NACIONALIDADE', 'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO',
       'SG_UF_NASCIMENTO', 'NM_MUNICIPIO_NASCIMENTO',
       'CD_MUNICIPIO_NASCIMENTO', 'TP_AGREMIACAO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'COMP_COLIGACAO', 'QT_PART_COL', 'QT_BENS',
       'VL_TOT_BENS', 'QT_VAGAS', 'REC_PROP', 'REC_PF', 'REC_PJ',
       'REC_PARTCAND', 'REC_OUTRAS', 'VR_DESPESA', 'VOTOS_PART_ELEC_ANT',
       'QT_ELEITOS_ELEC_ANT', 'RESULTADO'],
      dtype='object')

#### Análise das colunas / preditoras
- SG_UE: ok
- NM_UE: redundante
- NM_CANDIDATO: apenas para fins de identificação, sem aparente de predição
- SQ_CANDIDATO: chave/index
- NR_CANDIDATO: apenas para fins de identificação, sem fim aparente de predição
- NR_CPF_CANDIDATO: apenas para fins de identificação, sem fim aparente de predição
- NM_URNA_CANDIDATO: redundante
- CD_SITUACAO_CANDIDATURA: em tese, todos teriam o mesmo código. não parece ser preditora útil, excluir
- DS_SITUACAO_CANDIDATURA: redundante
- NR_PARTIDO: redundante
- SG_PARTIDO: ok
- NM_PARTIDO: redundante
- CD_OCUPACAO: ok
- DS_OCUPACAO: redundante
- DT_NASCIMENTO: redundante
- NR_TITULO_ELEITORAL_CANDIDATO: apenas para fins de identificação, sem fim aparente de predição
- IDADE_DATA_POSSE: ok
- CD_GENERO: ok
- DS_GENERO: redundante
- CD_GRAU_INSTRUCAO: ok
- DS_GRAU_INSTRUCAO: redundante
- CD_ESTADO_CIVIL: ok
- DS_ESTADO_CIVIL: redundante
- CD_NACIONALIDADE: ok
- DS_NACIONALIDADE: redundante
- CD_SIT_TOT_TURNO: flag
- DS_SIT_TOT_TURNO: flag
- SG_UF_NASCIMENTO: ok
- NM_MUNICIPIO_NASCIMENTO: redundante
- CD_MUNICIPIO_NASCIMENTO: ok
- TP_AGREMIACAO: ok
- SQ_COLIGACAO: redundante
- NM_COLIGACAO: redundante
- COMP_COLIGACAO: ok
- QT_PART_COL: ok
- QT_BENS: ok
- VL_TOT_BENS: ok
- QT_VAGAS: sem função preditora, mas é um limitante para a quantidade de positivos
- REC_PROP: ok
- REC_PF: ok
- REC_PJ: ok
- REC_PARTCAND: ok
- REC_OUTRAS: ok
- VR_DESPESA: ok
- VOTOS_PART_ELEC_ANT: ok (total de votos do partido para o cargo em disputa no pleito anterior)
- QT_ELEITOS_ELEC_ANT: ok (quantidade de vagas conquistadas pelo partido no pleito anterior)
- RESULTADO: target

In [213]:
# definindo colunas do primeiro dataset
cols = ['SQ_CANDIDATO','NM_UE','SG_PARTIDO','DS_OCUPACAO', 
        'IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO', 
        'DS_ESTADO_CIVIL', 'DS_NACIONALIDADE','SG_UF_NASCIMENTO', 
        'CD_MUNICIPIO_NASCIMENTO', 'TP_AGREMIACAO', 'COMP_COLIGACAO', 
        'QT_PART_COL', 'QT_BENS','VL_TOT_BENS', 
        'REC_PROP','REC_PF','REC_PJ','REC_PARTCAND','REC_OUTRAS', 
        'VR_DESPESA','VOTOS_PART_ELEC_ANT','QT_ELEITOS_ELEC_ANT', 'QT_VAGAS','DS_SIT_TOT_TURNO','RESULTADO']
base = rv08[cols].copy()

In [214]:
# prepara base para machine learning
# usa colunas com codificação, ao invés das colunas com descrição
# trata colunas categoricas que não tinha codificação ('uf', 'TP_AGREMIACAO' e 'COMP_COLIGACAO')

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

dict_uf = {'RO':11, 'AC':12, 'AM':13, 'RR':14, 'PA':15,
           'AP':16, 'TO':17, 'MA':21, 'PI':22, 'CE':23,
           'RN':24, 'PB':25, 'PE':26, 'AL':27, 'SE':28,
           'BA':29, 'MG':31, 'ES':32, 'RJ':33, 'SP':35,
           'PR':41, 'SC':42, 'RS':43, 'MS':50, 'MT':51,
           'GO':52, 'DF':53}
dict_agr = {'PARTIDO ISOLADO':0, 'COLIGACAO':1}


# definindo colunas do primeiro dataset
cols = ['SQ_CANDIDATO','SG_UE','NR_PARTIDO','CD_OCUPACAO', 
        'IDADE_DATA_POSSE', 'CD_GENERO', 'CD_GRAU_INSTRUCAO', 
        'CD_ESTADO_CIVIL', 'CD_NACIONALIDADE','SG_UF_NASCIMENTO', 
        'CD_MUNICIPIO_NASCIMENTO', 'TP_AGREMIACAO', 'COMP_COLIGACAO', 
        'QT_PART_COL', 'QT_BENS','VL_TOT_BENS', 
        'REC_PROP','REC_PF','REC_PJ','REC_PARTCAND','REC_OUTRAS', 'VR_DESPESA',
        'VOTOS_PART_ELEC_ANT','QT_ELEITOS_ELEC_ANT','RESULTADO']
base_ml = rv08[cols].copy()
base_ml['TP_AGREMIACAO'].replace(dict_agr, inplace=True)
base_ml['SG_UF_NASCIMENTO'].replace(dict_uf, inplace=True)
base_ml['COMP_COLIGACAO'] = le.fit_transform(base_ml.COMP_COLIGACAO)
base_ml.set_index('SQ_CANDIDATO', inplace=True)
# armazena arquivo csv pronto para Machine Learning
base_ml.to_csv('base_para_ml_cod_2012.csv')

In [195]:
base_ml.sample(5)

,SG_UE,NR_PARTIDO,CD_OCUPACAO,IDADE_DATA_POSSE,CD_GENERO,CD_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,CD_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,TP_AGREMIACAO,COMP_COLIGACAO,QT_PART_COL,QT_BENS,VL_TOT_BENS,REC_PROP,REC_PF,REC_PJ,REC_PARTCAND,REC_OUTRAS,VR_DESPESA,VOTOS_PART_ELEC_ANT,QT_ELEITOS_ELEC_ANT,RESULTADO
SQ_CANDIDATO,,,,,,,,,,,,,,,,,,,,,,,,
250000039073,71072,14,581,45,4,6,3,1,25,20516,1,7,2,0,0.00,0.00,0.00,0.00,"14,530.00",0.00,"5,030.00",305236,3,0
250000017640,71072,12,235,52,4,8,3,1,35,71072,0,0,1,2,"61,000.00",0.00,0.00,0.00,0.00,0.00,0.00,143901,1,0
250000018963,71072,50,237,42,2,6,3,1,23,14877,1,1,2,5,"164,100.00",0.00,0.00,600.00,289.00,0.00,889.00,36682,0,0
250000039103,71072,14,257,45,2,5,9,1,35,71072,1,7,2,12,"428,905.47",0.00,0.00,0.00,0.00,0.00,0.00,305236,3,0
250000018991,71072,50,169,37,2,7,1,1,35,71072,1,1,2,0,0.00,600.00,0.00,0.00,289.00,0.00,889.00,36682,0,0
